In [6]:
"""Practice notebook using https://www.kaggle.com/pierremegret/gensim-word2vec-tutorial to learn Word2Vec and additional word processing
techniques""" 


import re  # For preprocessing
import pandas as pd  # For data handling
from time import time  # To time our operations
from collections import defaultdict  # For word frequency

import spacy  # For preprocessing
from spacy.lang.en import English

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

#word2vec stuff here
import multiprocessing

from gensim.models import Word2Vec

from gensim.models.phrases import Phrases, Phraser  #for bigrams/common phrase ID


def cleaning(doc):
    #removes stopwords, easy to modifiy and lemmatize
    # doc needs to be a spacy Doc object
    txt = [token.lemma_ for token in doc if not token.is_stop]      #easy enough to stop lemmatizing by removing .lemma_ from token.lemma_
    # Word2Vec uses context words to learn the vector representation of a target word,
    # if a sentence is only one or two words long,
    # the benefit for the training is very small
    if len(txt) > 2:
        return ' '.join(txt)



In [9]:

t = time()
rough_df = pd.read_csv(r"C:\Users\bowri\square1\square2\dialogue-lines-of-the-simpsons\simpsons_dataset.csv")
print("size of dataset: ", rough_df.shape)  #prints size of matrix
#rough_data.head()
rough_df.isnull().sum()  #sum of null values, from pandas 

rough_df = rough_df.dropna().reset_index(drop=True) #removes null values from dataframe
rough_df.isnull().sum()

nlp = spacy.load('en_core_web_sm', disable=["parser", "ner"])
#nlp = spacy.load("en_core_web_sm", disable=["parser", "tagger"]) r"C:\Users\bowri\Anaconda3\Lib\site-packages\en_core_web_sm
#nlp = spacy.load('en_core_web_sm')

brief_cleaning = (re.sub("[^A-Za-z']+", ' ', str(row)).lower() for row in rough_df['spoken_words']) #removing non-alphabetical char w REGEX




txt = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=5000, n_threads=-1)]

print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))

#everything in list at this point, txt

df_clean = pd.DataFrame({'clean': txt})
df_clean = df_clean.dropna().drop_duplicates()
df_clean.shape

#now in a clean dataframe with dropped duplibcations

sent = [row.split() for row in df_clean['clean']]              #to ID common phrases? idk about benefit here, 
phrases = Phrases(sent, min_count=30, progress_per=10000)   #also sent is a list here 

bigram = Phraser(phrases)
sentences = bigram[sent]
print("sentences is type ", type(sentences))


size of dataset:  (158314, 2)
Time to clean up everything: 1.19 mins


INFO - 10:51:33: collecting all words and their counts
INFO - 10:51:33: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 10:51:33: PROGRESS: at sentence #10000, processed 63561 words and 52816 word types
INFO - 10:51:34: PROGRESS: at sentence #20000, processed 130943 words and 99866 word types
INFO - 10:51:34: PROGRESS: at sentence #30000, processed 192972 words and 138532 word types
INFO - 10:51:34: PROGRESS: at sentence #40000, processed 249842 words and 172659 word types
INFO - 10:51:34: PROGRESS: at sentence #50000, processed 311265 words and 208566 word types
INFO - 10:51:34: PROGRESS: at sentence #60000, processed 373588 words and 243702 word types
INFO - 10:51:34: PROGRESS: at sentence #70000, processed 436441 words and 278740 word types
INFO - 10:51:34: PROGRESS: at sentence #80000, processed 497829 words and 311886 word types
INFO - 10:51:34: collected 330804 word types from a corpus of 537160 words (unigram + bigrams) and 85964 sentences
INFO - 10:51:34: us

sentences is type  <class 'gensim.interfaces.TransformedCorpus'>


In [13]:
#Word to Vec stuff begins here full
cores = multiprocessing.cpu_count() # Count the number of cores in a computer, it's 8

#1) INITALIZING THE MODEL
w2v_model = Word2Vec(min_count=20,         #Ignores all words with total absolute frequency lower than this - (2, 100)
                     window=2,           #The maximum distance between the current and predicted word within a sentence.
                     size = 300,           #size of feature vectors
                     sample=6e-5,   #threshold for configuring which higher-frequency words are randomly downsampled. influencial. (0, 1e-5)
                     alpha=0.03,        #learning rate initial (0.01 - 0.05)
                     min_alpha=0.0007,     #final learning rate, linearly drops to this as training goes
                     negative=20,  # If > 0,  how many "noise words" should be drown. If set to 0, no negative sampling is used. - (5, 20)
                     workers=cores-2)        #how many cores doing the work


#2) BUILDING VOCAB

t = time()

w2v_model.build_vocab(sentences, progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

#3) TRAINING MODEL

#    total_examples = int, Count of sentences;
 #   epochs = int - Number of iterations (epochs) over the corpus - [10, 20, 30]
    
t = time()

w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))


w2v_model.init_sims(replace=True)  #do this when done training the model?

INFO - 11:22:22: collecting all words and their counts
INFO - 11:22:22: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 11:22:22: PROGRESS: at sentence #10000, processed 61718 words, keeping 9558 word types
INFO - 11:22:22: PROGRESS: at sentence #20000, processed 127351 words, keeping 14506 word types
INFO - 11:22:23: PROGRESS: at sentence #30000, processed 187829 words, keeping 17619 word types
INFO - 11:22:23: PROGRESS: at sentence #40000, processed 243332 words, keeping 20385 word types
INFO - 11:22:23: PROGRESS: at sentence #50000, processed 303182 words, keeping 22878 word types
INFO - 11:22:23: PROGRESS: at sentence #60000, processed 363940 words, keeping 25200 word types
INFO - 11:22:23: PROGRESS: at sentence #70000, processed 425408 words, keeping 27401 word types
INFO - 11:22:24: PROGRESS: at sentence #80000, processed 485464 words, keeping 29275 word types
INFO - 11:22:24: collected 30178 word types from a corpus of 523700 raw words and 85964 sentence

Time to build vocab: 0.03 mins


INFO - 11:22:24: training model with 6 workers on 3319 vocabulary and 300 features, using sg=0 hs=0 sample=6e-05 negative=20 window=2
INFO - 11:22:25: EPOCH 1 - PROGRESS: at 53.70% examples, 103594 words/s, in_qsize 0, out_qsize 0
INFO - 11:22:26: worker thread finished; awaiting finish of 5 more threads
INFO - 11:22:26: worker thread finished; awaiting finish of 4 more threads
INFO - 11:22:26: worker thread finished; awaiting finish of 3 more threads
INFO - 11:22:26: worker thread finished; awaiting finish of 2 more threads
INFO - 11:22:26: worker thread finished; awaiting finish of 1 more threads
INFO - 11:22:26: worker thread finished; awaiting finish of 0 more threads
INFO - 11:22:26: EPOCH - 1 : training on 523700 raw words (198820 effective words) took 1.9s, 102725 effective words/s
INFO - 11:22:27: EPOCH 2 - PROGRESS: at 43.67% examples, 84641 words/s, in_qsize 0, out_qsize 0
INFO - 11:22:28: EPOCH 2 - PROGRESS: at 97.45% examples, 95585 words/s, in_qsize 0, out_qsize 0
INFO - 1

Time to train the model: 1.14 mins


INFO - 11:23:32: precomputing L2-norms of word weight vectors


In [14]:
#testing
w2v_model.wv.most_similar(positive=["homer"])

[('bongo', 0.7788041830062866),
 ('marge', 0.7733016014099121),
 ('gee', 0.7590972185134888),
 ('crummy', 0.753426194190979),
 ('wife', 0.7501407861709595),
 ('rude', 0.7463955879211426),
 ('snuggle', 0.7443416714668274),
 ('sweetheart', 0.7426704168319702),
 ('sorry', 0.7412137985229492),
 ('worry', 0.7329186201095581)]

In [34]:
"""Add some docstrings later"""


import re  # For preprocessing
import pandas as pd  # For data handling
from time import time  # To time our operations
from collections import defaultdict  # For word frequency
import spacy  # For preprocessing
from spacy.lang.en import English  #not sure what this is for, check if needed
import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

#word2vec stuff here
import multiprocessing
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases, Phraser  #for bigrams/common phrase ID
#import json_read_clean as jRC 

#TO-DO:
#1) add some default variables, especially for adapting the word2vec model here
#2) document
#3) Test various parameters for word2vec functions


def word2vec_book_model(list_of_book_words):

    phrases = Phrases(list_of_book_words, min_count=30, progress_per=10000)   #also sent is a list here 
    bigram = Phraser(phrases)
    sentences = bigram[list_of_book_words]
    
    cores = multiprocessing.cpu_count() # Count the number of cores in a computer, it's 8

#1) INITALIZING THE MODEL
    w2v_model = Word2Vec(min_count=2,         #Ignores all words with total absolute frequency lower than this - (2, 100)
                     window=2,           #The maximum distance between the current and predicted word within a sentence.
                     size = 300,           #size of feature vectors
                     sample=6e-5,   #threshold for configuring which higher-frequency words are randomly downsampled. influencial. (0, 1e-5)
                     alpha=0.03,        #learning rate initial (0.01 - 0.05)
                     min_alpha=0.0007,     #final learning rate, linearly drops to this as training goes
                     negative=0,  # If > 0,  how many "noise words" should be drown. If set to 0, no negative sampling is used. - (5, 20)
                     workers=cores-2)        #how many cores doing the work

#2) BUILDING VOCAB
    t = time()
    w2v_model.build_vocab(sentences, progress_per=10000)
    print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))
    
#3) TRAINING MODEL

    #    total_examples = int, Count of sentences;
     #   epochs = int - Number of iterations (epochs) over the corpus - [10, 20, 30]
    
    t = time()
    w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)
    print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))


    w2v_model.init_sims(replace=True)  #do this when done training the model?
    
    print("the size of the vocabulary is: ", len(w2v_model.wv.vocab))
    print(w2v_model.wv.most_similar(positive=["carbon"]))



In [30]:
test_schaller_list = ['thumbnail', 'space-filling', 'model', 'section', 'polyethylene', 'terephthalate', 'polymer', 'also', 'known', 'pet', 'pete', 'polyester', 'used', 'plastic', 'bottles', 'color', 'code', 'carbon', 'c', '(black)', 'hydrogen', 'h', '(white)', 'oxygen', 'o', '(red)', 'image', 'used', 'permission', '(public', 'domain', 'jynto)', 'the', 'first', 'synthetic', 'polymer', 'really', 'seize', 'publics', 'consciousness', '--', 'first', 'one', 'change', 'course', 'world', 'events', '--', 'nylon', 'not', 'long', 'discovery', '1930s', 'supplanted', 'silk', 'material', 'choice', 'stockings', 'transformed', 'luxury', 'item', 'widely', 'available', 'commodity', 'nylon', 'came', 'time', 'second', 'world', 'war', 'soon', 'parachutes', 'went', 'made', 'silk', 'fabricated', 'nylon', 'the', 'development', 'nylon', 'took', 'place', 'du', 'ponts', 'experimental', 'research', 'station', 'delaware', 'for', 'much', 'twentieth', 'century', 'du', 'pont', 'large', 'companies', 'relied', 'pursuit', 'basic', 'science', 'engine', 'future', 'development', 'knowledge', 'infrastructure', 'made', 'future', 'products', 'possible', 'scientists', 'asked', 'explore', 'unknown', 'applications', 'work', 'would', 'follow', 'naturally', 'when', 'wallace', 'carrothers', 'du', 'pont', 'developed', 'nylon', 'trying', 'explore', 'ideas', 'proposed', 'hermann', 'staudinger', 'eth', 'zurich', 'these', 'ideas', 'initially', 'controversial', 'held', 'many', 'materials', 'around', 'us', 'including', 'things', 'like', 'cotton', 'spider', 'silk', 'made', 'small', 'molecular', 'units', 'covalently', 'bonded', 'these', 'little', 'molecules', 'monomers', 'bind', 'together', 'form', 'immensely', 'long', 'chains', 'in', 'monomers', 'lose', 'individual', 'identities', 'simply', 'become', 'repeating', 'units', 'long', 'chain', 'they', 'come', 'together', 'form', 'polymers', 'nylon', 'particular', 'example', 'polyamide', 'it', 'composed', 'long', 'chains', 'contain', 'regularly', 'repeating', 'amide', 'bonds', 'n-co', 'thats', 'motif', 'found', 'proteins', 'biology', 'pattern', 'found', 'spiders', 'silk', 'composed', 'protein', 'conceptually', 'taught', 'biology', 'class', 'proteins', 'made', 'amino', 'acids', 'an', 'amide', 'bond', '(or', 'peptide', 'bond)', 'occurs', 'nitrogen', 'amine', 'brought', 'together', 'carbon', 'carboxylic', 'acid', 'a', 'molecule', 'water', 'lost', 'nitrogen', 'takes', 'place', 'oxygen', 'carboxylic', 'acid', 'forming', 'amide', 'instead', 'the', 'loss', 'water', 'leads', 'term', 'condensation', 'reaction', 'early', 'studies', 'reactions', 'led', 'water', 'forming', 'droplets', 'laboratory', 'glassware', 'bubbled', 'reaction', 'in', 'practice', 'industrial', 'nylon', 'production', 'really', 'complicated', 'of', 'course', 'old', 'amide', 'become', 'protein', 'what', 'makes', 'amino', 'acids', 'special', 'fact', 'difunctional', 'they', 'contain', 'one', 'functional', 'group', '(an', 'amine', 'say', 'carboxylic', 'acid)', 'two', 'so', 'amine', 'group', 'amino', 'acid', 'bonds', 'carbonyl', 'neighbour', 'carboxyl', 'group', 'bond', 'amine', 'different', 'neighbour', 'its', 'like', 'two', 'hands', 'hold', 'onto', 'friend', 'hand', 'friends', 'hold', 'onto', 'another', 'forming', 'chain', 'this', 'functionality', 'key', 'part', 'polymers', 'form', 'because', 'difunctional', 'molecules', 'form', 'bonds', 'two', 'directions', 'simple', 'coupling', 'reaction', '(an', 'amine', 'plus', 'carboxylic', 'acid', 'making', 'amide', 'two', 'small', 'molecules', 'making', 'another', 'small', 'molecule)', 'becomes', 'polymerization', '(many', 'small', 'molecules', 'forming', 'enormous', 'molecule)', 'well', 'nylon', 'isnt', 'made', 'amino', 'acid', 'although', 'coupling', 'reaction', 'similar', 'instead', 'made', 'two', 'different', 'molecules', 'difunctional', 'diamine', 'dicarboxylic', 'acid', 'the', 'diamine', 'bonds', 'neighbouring', 'carbonyl', 'end', 'dicarboxylic', 'acid', 'bonds', 'neighbouring', 'amine', 'end', 'for', 'common', 'material', 'two', 'reactants', 'six', 'carbons', 'long', 'giving', 'rise', 'term', 'nylon', '66.', 'nylon', '66', 'example', 'something', 'called', 'alternating', 'co-polymer', 'it', 'isnt', 'one', 'unit', 'repeating', 'along', 'chain', 'two', 'the', 'two', 'monomers', 'make', 'polymer', 'together', 'we', 'sometimes', 'say', 'monomers', 'enchained', 'become', 'part', 'polymer', 'and', 'course', 'alternate', 'along', 'chain', 'one', 'bind', 'complementary', 'in', 'drawings', 'dashed', 'lines', 'meant', 'suggest', 'continuity', 'pattern', 'shown', 'drawing', 'keeps', 'repeating', 'right', 'left', 'more', 'commonly', 'polymers', 'drawn', 'using', 'parenthetical', 'notation', 'below', 'part', 'shown', 'parentheses', 'keeps', 'repeating', 'if', 'could', 'make', 'stamp', 'picture', 'could', 'construct', 'polymer', 'chain', 'simply', 'stamping', 'image', 'across', 'sheet', 'paper', 'carrothers', 'development', 'nylon', 'du', 'pont', 'led', 'indirectly', 'similar', 'polymer', 'made', 'different', 'way', 'across', 'atlantic', 'ocean', 'paul', 'schlack', 'ig', 'farben', 'germany', 'looking', 'way', 'make', 'similar', 'material', 'would', 'subject', 'du', 'ponts', 'patent', 'his', 'efforts', 'led', 'material', 'called', 'perlon', 'sometimes', 'referred', 'nylon', '6', 'nylon', '6', 'alternating', 'co-polymer', 'like', 'nylon', '66', 'its', 'polymer', 'and', 'isnt', 'made', 'difunctional', 'monomer', 'like', 'nylon', '66', 'proteins', 'instead', 'made', 'cyclic', 'amide', 'sometimes', 'called', 'lactam', 'to', 'polymerize', 'lactam', 'break', 'open', 'linear', 'form', 'lactam', 'monomers', 'end', 'enchained', 'head-to-tail', 'this', 'process', 'called', 'ring-opening', 'polymerization', 'there', 'number', 'things', 'different', 'two', 'ways', 'making', 'nylon', 'one', 'turns', 'whole', 'concept', 'synthesis', 'synthesis', 'act', 'making', 'things', 'its', 'little', 'like', 'cooking', 'you', 'gather', 'ingredients', 'you', 'take', 'right', 'amount', 'right', 'amount', 'you', 'mix', 'together', 'get', 'something', 'wonderful', 'but', 'thats', 'difference', 'making', 'nylon', '66', 'carrothers', 'mixed', 'two', 'different', 'compounds', 'together', 'he', 'poured', 'one', 'compound', 'one', 'bottle', 'another', 'compound', 'another', 'bottle', 'he', 'mixed', 'together', 'got', 'polymer', 'in', 'contrast', 'schlack', 'didnt', 'mix', 'anything', 'together', 'least', 'far', 'seen', 'far', 'it', 'came', 'one', 'bottle', 'so', 'wasnt', 'already', 'nylon', '6', 'bottle', 'if', 'monomers', 'react', 'couldnt', 'gone', 'ahead', 'done', 'bottle', 'ring-opening', 'polymerization', 'least', 'context', 'example', 'chain', 'reaction', 'chain', 'reactions', 'dont', 'happen', 'need', 'jump', 'start', 'to', 'understand', 'appreciate', 'two', 'materials', 'come', 'two', 'different', 'classes', 'polymerization', 'reactions', 'nylon', '66', 'product', 'condensation', 'reaction', 'an', 'amine', 'mixed', 'carboxylic', 'acid', 'water', 'released', 'amide', 'bond', 'formed', 'even', 'pair', 'reactants', 'chosen', 'synthetically', 'efficient', '--', 'say', 'amine', 'acid', 'chloride', 'instead', 'carboxylic', 'acid', '--', 'condensation', 'reaction', 'still', 'results', 'case', 'releasing', 'hydrogen', 'chloride', 'nylon', '6', 'hand', 'product', 'condensation', 'reaction', '--', 'least', 'first', 'glance', 'look', 'carefully', 'monomer', 'polymer', 'count', 'atoms', 'theres', 'nothing', 'missing', 'no', 'molecule', 'water', 'hydrogen', 'chloride', 'anything', 'else', 'released', 'the', 'nitrogen', 'one', 'end', 'chain', 'simply', 'attaches', 'carbonyl', 'next', 'but', 'walk', 'end', 'chain', 'reach', 'last', 'carbonyl', 'find', 'theres', 'nitrogen', 'attached', 'that', 'nitrogen', 'nucleophile', 'bound', 'carbonyl', 'next', 'monomer', 'there', 'something', 'though', 'carbon', 'cant', 'sitting', 'three', 'bonds', 'so', 'must', 'nucleophile', 'added', 'carbonyl', 'springing', 'loose', 'amine', 'added', 'next', 'carbonyl', 'springing', 'next', 'amine', 'there', 'original', 'nucleophile', 'this', 'jump', 'start', 'chain', 'reaction', 'needed', 'a', 'nucleophile', 'added', 'get', 'things', 'going', 'once', 'reacted', 'first', 'monomer', 'amine', 'became', 'nucleophile', 'next', 'monomer', 'ring-opening', 'produced', 'another', 'amine', 'nucleophile', 'because', 'always', 'new', 'nucleophile', 'produced', 'next', 'ring', 'opens', 'reaction', 'keeps', 'going', 'that', 'first', 'nucleophile', 'one', 'got', 'everything', 'started', 'called', 'initiator', 'the', 'need', 'initiator', 'hallmark', 'chain', 'reactions', 'in', 'addition', 'initiator', 'ring-opening', 'polymerizations', 'frequently', 'employ', 'catalysts', 'accelerate', 'reaction', 'condensation', 'polymerizations', 'the', 'catalyst', 'may', 'lewis', 'acid', 'activates', 'carbonyl', 'organocatalyst', 'thing', 'via', 'hydrogen', 'bonding', 'both', 'catalytic', 'approaches', 'also', 'make', 'use', 'nucleophilic', 'catalysis', 'in', 'case', 'nucleophilicity', 'nucleophile', 'may', 'enhanced', 'else', 'temporary', 'nucleophile', 'may', 'add', 'carbonyl', 'replaced', 'nucleophilic', 'group', 'resulting', 'enchainment', 'other', 'polyamides', 'also', 'made', 'ring-opening', 'cyclic', 'amides', 'the', 'approach', 'also', 'used', 'prepare', 'polyesters', 'cyclic', 'esters', 'also', 'called', 'lactones', 'in', 'case', 'reaction', 'sometimes', 'called', 'ring-opening', 'trans-esterification', 'polymerization', 'rotep', 'short', 'polyolefins', 'made', 'olefins,', 'may', 'know', 'alkenes', 'olefin', 'older', 'term', 'alkene', 'still', 'commonly', 'used', 'industry', 'these', 'compounds', 'make', 'significant', 'fraction', 'commercially-used', 'polymers', 'today', 'if', 'think', 'common', 'recyclable', 'plastics', 'polyethylene', '(2', '4', 'depending', 'material', 'made)', 'poly(vinyl', 'chloride)', '(3)', 'polypropylene', '(5)', 'polystyrene', '(6)', 'examples', 'polyolefins', 'polyolefins', 'known', 'time', 'although', 'took', 'century', 'time', 'first', 'documented', 'recognized', 'polymers', 'german', 'chemists', 'mid-1800s', 'aware', 'certain', 'tree', 'resins', 'gave', 'rise', 'hard', 'materials', 'time', 'materials', 'included', 'polystyrene', 'if', 'compare', 'repeating', 'structure', 'polystyrene', 'structure', 'styrene', 'imagine', 'polystyrene', 'made', 'series', 'styrene', 'molecules', 'strung', 'together', 'essentially', 'double', 'bond', 'styrene', 'moved', 'attach', 'next', 'molecule', 'double', 'bond', 'done', 'thing', 'polyolefins', 'formed', 'way', 'wide', 'variety', 'alkenes', 'leading', 'array', 'different', 'materials', 'properties', 'suited', 'unique', 'applications', 'note', 'like', 'ring-opening', 'polymerization', 'olefin', 'polymerization', 'depends', 'molecules', 'reacting', 'molecules', 'like', 'that', 'isnt', 'likely', 'happen', 'need', 'something', 'come', 'along', 'induce', 'reaction', 'olefin', 'polymerization', 'thus', 'another', 'case', 'monomers', 'tied', 'together', 'chain', 'reaction', 'the', 'reaction', 'require', 'initiator', 'get', 'things', 'started', 'that', 'initiator', 'likely', 'become', 'end', 'group', 'hanging', 'one', 'end', 'polymer', 'chain', 'polyolefins', 'usually', 'made', 'olefins', 'tethering', 'one', 'alkene', 'unit', 'next', 'trading', 'pi', 'bond', 'within', 'monomer', 'single', 'bond', 'two', 'repeat', 'units', 'however', 'another', 'approach', 'converts', 'cyclic', 'alkenes', 'polymers', 'this', 'approach', 'reminiscent', 'ring-opening', 'cyclic', 'esters', 'amides', 'for', 'example', 'imagine', 'cyclopentene', 'ring', 'opening', 'double', 'bond', 'reaching', 'join', 'rings', 'either', 'side', 'a', 'series', 'cyclopentene', 'rings', 'joined', 'together', 'row', 'would', 'look', 'something', 'like', 'we', 'would', 'probably', 'draw', 'usual', 'zig-zag', 'conformation', 'this', 'act', 'taking', 'cyclic', 'alkene', 'splitting', 'open', 'double', 'bond', 'knitting', 'together', 'alkenes', 'long', 'chain', 'called', 'ring-opening', 'metathesis', 'polymerization', 'sometimes', 'called', 'romp', 'short', 'it', 'things', 'common', 'ring-opening', 'polymerizations', 'rotep', 'in', 'rotep', 'romp', 'like', 'molecules', 'react', 'together', 'form', 'polymer', 'in', 'rotep', 'would', 'two', 'cyclic', 'esters', 'in', 'romp', 'would', 'two', 'cyclic', 'alkenes', 'that', 'different', 'condensation', 'polymerization', 'required', 'two', 'complementary', 'molecules', 'difunctional', 'amine', 'difunctional', 'acid', 'chloride', 'partly', 'consequence', 'like', 'molecules', 'reacting', 'together', 'rotep', 'romp', 'chain', 'reactions', 'in', 'order', 'get', 'molecules', 'react', 'need', 'initiator', 'the', 'initiator', 'jump-starts', 'reaction', 'other', 'olefin', 'polymerizations', 'followed', 'pattern', 'well', 'alkenes', 'dont', 'normally', 'react', 'together', '(there', 'circumstances', 'neednt', 'get', 'events', 'dont', 'usually', 'anything', 'polymerization)', 'olefin', 'polymerizations', 'general', 'go', 'chain', 'reactions', 'require', 'initiator', 'get', 'started', 'despite', 'similarities', 'rotep', 'romp', 'reactions', 'actually', 'quite', 'different', 'occur', 'regular', 'olefin', 'polymerizations', 'conditions', 'required', 'initiate', 'polymerization', 'unique', 'case', 'romp', 'requires', 'something', 'called', 'olefin', 'metathesis', 'catalyst', 'an', 'olefin', 'metathesis', 'catalyst', 'transition', 'metal', 'compound', 'capable', 'splitting', 'double', 'bond', 'alkene', 'half', 'putting', 'two', 'pieces', 'together', 'alkenes', 'the', 'key', 'part', 'olefin', 'metathesis', 'catalyst', 'metal-carbon', 'double', 'bond', 'that', 'group', 'capable', 'switching', 'ends', 'alkenes', 'around', 'different', 'partners', 'polymers', 'long-chain', 'molecules', 'formed', 'individual', 'molecular', 'building', 'blocks', 'typically', 'building', 'blocks', 'organic', 'molecules', 'held', 'together', 'via', 'covalent', 'bonds', 'what', 'kinds', 'building', 'blocks', 'available', 'the', 'formation', 'coordination', 'compounds', 'one', 'many', 'important', 'aspects', 'inorganic', 'chemistry', 'in', 'coordination', 'compound', 'electron', 'pair', 'donor', 'called', 'ligand', 'shares', 'electron', 'pair', 'metal', 'atom', 'frequently', 'metal', 'atom', 'transition', 'metal', 'commonly', 'transition', 'metal', 'ion', 'examples', 'well', 'for', 'example', 'nitrogen', 'pyridine', 'ring', 'lone', 'pair', 'pyridine', 'act', 'ligand', 'lone', 'pair', 'shared', 'metal', 'center', 'vanadium', 'trichloro(oxo)vanadium', 'the', 'lone', 'pair', 'becomes', 'nitrogen-vanadium', 'bond', 'sometimes', 'bond', 'drawn', 'short', 'arrow', 'lone', 'pair', 'vanadium', 'emphasizing', 'origin', 'often', 'simply', 'drawn', 'line', 'like', 'bond', 'one', 'interesting', 'things', 'metal', 'atoms', 'capacity', 'form', 'variable', 'numbers', 'bonds', 'although', 'palladium', 'dichloride', 'could', 'form', 'coordination', 'complex', 'binding', 'one', 'pyridine', 'ligand', 'also', 'two', 'pyridines', 'in', 'former', 'case', 'would', 'form', 'three-coordinate', 'complex', 'latter', 'case', 'would', 'lead', 'four-coordinate', 'palladium', 'compound', 'remember', 'ability', 'bond', 'two', 'neighbouring', 'groups', 'rather', 'one', 'important', 'feature', 'allow', 'small', 'molecule', 'become', 'enchained', 'polymer', 'the', 'palladium', 'link', 'chain', 'three', 'units', 'long', 'what', 'ligand', 'also', 'capacity', 'bind', 'two', 'things', 'what', 'instead', 'pyridine', 'ligand', '44-bipyridine', 'that', 'molecule', 'could', 'bind', 'palladium', 'atom', 'either', 'end', 'once', 'bpy', '(thats', 'abbreviation', 'bipyridine', 'pronounced', 'bippy)', 'bound', 'palladium', 'atom', 'would', 'still', 'second', 'nitrogen', 'lone', 'pair', 'could', 'use', 'bind', 'another', 'the', 'palladium', 'would', 'free', 'bind', 'second', 'bpy', 'as', 'result', 'two', 'monomeric', 'units', 'able', 'form', 'alternating', 'chain', 'why', 'would', 'people', 'want', 'the', 'general', 'idea', 'exploit', 'properties', 'metal', 'atoms', 'new', 'ways', 'what', 'metals', 'good', 'they', 'good', 'conducting', 'electricity', 'sometimes', 'hold', 'useful', 'magnetic', 'properties', 'if', 'properties', 'incorporated', 'material', 'like', 'organic', 'polymers', 'tend', 'inexpensive', 'lightweight', 'maybe', 'used', 'make', 'materials', 'sorts', 'applications', 'everyday', 'life', 'metals', 'also', 'good', 'catalysts', 'wide', 'range', 'reactions', 'a', 'coordination', 'polymer', 'serves', 'space', 'metal', 'atoms', 'evenly', 'within', 'structure', 'lot', 'surface', 'area', 'could', 'promote', 'catalytic', 'efficiency', 'all', 'potential', 'uses', 'driven', 'great', 'deal', 'research', 'coordination', 'polymers', 'recent', 'years', '(as', 'well', 'related', 'metal-organic', 'frameworks)', 'now', 'bonds', 'along', 'metal-ligand', 'chain', 'really', 'hold', 'together', 'form', 'polymer', 'one', 'key', 'difference', 'covalent', 'bonds', 'seen', 'organic', 'polymers', 'dative', 'bonds', 'dative', 'bonds', 'reversible', 'there', 'always', 'equilibrium', 'metal-ligand', 'complex', 'free', 'metal', 'ligand', 'of', 'course', 'bond', 'might', 'strong', 'case', 'equilibrium', 'lies', 'toward', 'metal-ligand', 'complex', 'in', 'words', 'large', 'fraction', 'material', 'would', 'form', 'metal-ligand', 'complex', 'how', 'large', 'fraction', 'the', 'real', 'value', 'isnt', 'important', 'moment', 'for', 'purposes', 'lets', 'say', 'put', 'metal', 'ligand', 'together', '90', 'molecules', 'form', 'complex', 'and', 'suppose', 'thats', 'also', 'true', 'next', 'step', 'bringing', 'another', 'ligand', 'picture', 'bind', 'metal', 'step', 'bringing', 'another', 'metal', 'bind', 'end', 'one', 'ligands', 'so', 'suppose', 'events', 'leads', '90', 'product', 'formation', 'just', 'getting', 'four-unit', 'chain', 'relies', 'three', 'different', 'equilibria', 'if', 'say', 'step', 'proceeds', '90', 'way', '(which', 'sounds', 'pretty', 'good', 'would', 'happy', '90', 'yield', 'reaction', 'lab)', 'entire', 'three-step', 'process', 'would', 'yield', '090', 'x', '090', 'x', '090', '073', '73', 'product', 'not', 'bad', 'but', 'useful', 'polymer', 'chains', 'might', 'thousand', 'units', 'long', 'case', 'amount', 'polymer', 'actually', 'formed', 'proper', 'chain', 'length', 'would', '(090)999', '194', 'x', '10-46', '194', 'x', '10-44', ',', 'thats', 'ridiculously', 'small', 'amount', 'now', 'real', 'calculation', 'equilibrium', 'concentration', 'course', 'sophisticated', 'quick', 'exercise', 'underscores', 'important', 'point', 'order', 'form', 'stable', 'coordination', 'polymer', 'appreciable', 'length', 'metal', 'must', 'bind', 'ligand', 'tightly', 'for', 'reason', 'many', 'approaches', 'coordination', 'polymers', 'employed', 'multidentate', 'ligands', 'course', 'bind', 'tightly', 'monodentate', 'ligands', 'for', 'example', 'polymer', 'formed', 'using', 'bidentate', 'binding', 'shown', 'would', 'much', 'stable', 'respect', 'depolymerization', '(ie', 'falling', 'apart', 'monomers', 'again)', 'monodentate', 'example', 'shown', 'earlier', 'so', 'far', 'seen', 'covalent', 'bonds', 'used', 'bind', 'monomers', 'together', 'longer', 'chains', 'forming', 'polymers', 'there', 'also', 'great', 'deal', 'interest', 'recently', 'using', 'intermolecular', 'attractions', 'make', 'similar', 'structures', 'of', 'course', 'intermolecular', 'attractions', 'important', 'forming', 'large', 'organized', 'structures', 'biology', 'think', 'twin', 'helices', 'held', 'together', 'dna', 'secondary', 'higher-order', 'structures', 'proteins', 'clearly', 'intermolecular', 'attractions', 'used', 'form', 'large', 'stable', 'structures', 'if', 'goal', 'form', 'chain', 'analogous', 'chains', 'monomers', 'seen', 'polymers', 'would', 'useful', 'two', 'sites', 'within', 'monomer', 'interact', 'monomers', 'such', 'molecule', 'would', 'able', 'hold', 'hands', 'two', 'partners', 'speak', 'if', 'take', 'hydrogen', 'bonding', 'example', 'strong', 'intermolecular', 'force', 'look', 'molecule', 'ethylene', 'glycol', '(also', 'called', '12-dihydroxyethane)', 'ethylene', 'glycol', 'looks', 'kind', 'like', 'ethanol', '(ch3ch2oh)', 'extra', 'oh', 'group', 'a', 'physical', 'sample', 'ethylene', 'glycol', 'colorless', 'liquid', 'like', 'ethanol', 'noticeable', 'differences', 'two', 'the', 'boiling', 'point', 'ethylene', 'glycol', 'around', '198°c', 'hundred', 'degrees', 'higher', 'ethanol', 'furthermore', 'ethanol', 'isnt', 'viscous', 'pours', 'quite', 'easily', 'tried', 'stir', 'pour', 'ethylene', 'glycol', 'would', 'notice', 'seems', 'much', 'thicker', 'something', 'holding', 'ethylene', 'glycol', 'molecules', 'tightly', 'together', 'it', 'isnt', 'hard', 'imagine', 'formation', 'chain', 'structures', 'involving', 'groups', 'molecules', 'if', 'look', 'diols', 'containing', 'longer', 'carbon', 'chains', 'even', 'easier', 'imagine', 'something', 'looks', 'kind', 'like', 'polymer', 'monomers', 'enchained', 'together', 'via', 'hydrogen', 'bonding', 'interactions', 'hydrogen', 'bonds', 'strong', 'still', 'intermolecular', 'attractions', 'rather', 'true', 'covalent', 'bonds', 'they', 'readily', 'reversible', 'chains', 'formed', 'via', 'interaction', 'diols', 'could', 'depolymerize', 'easily', 'individual', 'diols', 'could', 'readily', 'fall', 'end', 'chain', 'diol', 'chains', 'could', 'easily', 'break', 'somewhere', 'middle', 'what', 'tetraol', 'if', 'two', 'hydroxy', 'groups', 'either', 'end', 'chain', 'chain', 'would', 'much', 'less', 'likely', 'come', 'apart', 'even', 'one', 'hydrogen', 'bond', 'broke', 'one', 'linkages', 'would', 'still', 'second', 'one', 'holding', 'two', 'neighbors', 'together', 'a', 'molecule', 'like', 'could', 'form', 'much', 'stable', 'supramolecular', 'assembly', 'the', 'dynamic', 'nature', 'supramolecular', 'assemblies', 'part', 'makes', 'interesting', 'researchers', 'stable', 'structures', 'easily', 'form', 'one', 'set', 'circumstances', 'gently', 'come', 'apart', 'another', 'could', 'lots', 'applications', 'for', 'example', 'researchers', 'interested', 'developing', 'novel', 'materials', 'liquid', 'crystals', 'might', 'converted', 'one', 'form', 'another', 'using', 'different', 'stimuli', 'others', 'trying', 'develop', 'materials', 'useful', 'medicinal', 'therapeutic', 'applications', 'the', 'principle', 'functionality', 'means', 'almost', 'type', 'organic', 'reaction', 'could', 'potentially', 'used', 'make', 'polymers', 'for', 'instance', 'compound', 'two', 'functional', 'groups', 'kind', 'could', 'undergo', 'reaction', 'two', 'different', 'sites', 'forming', 'new', 'bonds', 'two', 'neighbors', 'the', 'compound', 'thereby', 'becomes', 'enchained', 'trio', 'formerly', 'independent', 'molecules', 'if', 'neighboring', 'molecules', 'also', 'difunctional', 'pattern', 'repeat', 'forming', 'polymer', 'take', 'diels', 'alder', 'reaction', 'illustrative', 'example', 'on', 'paper', 'reaction', 'fairly', 'straightforward', 'even', 'isnt', 'common', 'a', 'molecule', 'pair', 'conjugated', 'alkenes', 'diene', 'cyclizes', 'another', 'alkene', 'dienophile', 'form', 'new', 'six-membered', 'ring', 'cyclohexene', 'the', 'diene', 'needs', 'two', 'double', 'bonds', 'dienophile', 'needs', 'one', 'nevertheless', 'happens', 'dienophile', 'extra', 'double', 'bond', 'the', 'diels', 'alder', 'adduct', 'forms', 'would', 'end', 'two', 'double', 'bonds', 'one', 'formed', 'original', 'diene', 'always', 'leftover', 'dienophile', 'the', 'resulting', 'compound', 'difunctional', 'alkene', 'each', 'end', 'molecule', 'could', 'potentially', 'undergo', 'another', 'diels', 'alder', 'reaction', 'additional', 'diene', 'each', 'time', 'reaction', 'occurs', 'new', 'alkene', 'left', 'behind', 'diene', 'used', 'preserving', 'difunctionality', 'another', 'step', 'that', 'difunctionality', 'forms', 'basis', 'potential', 'polymer', 'chemistry', 'in', 'fact', 'diels', 'alder', 'reactions', 'exploited', 'researchers', 'number', 'ways', 'make', 'materials', 'useful', 'properties', 'the', 'fact', 'six-membered', 'rings', 'introduced', 'along', 'backbone', 'rather', 'chain', 'single', 'bonds', 'means', 'materials', 'display', 'varying', 'levels', 'conformational', 'rigidity', 'resulting', 'unique', 'properties', 'polymers', 'large', 'molecules', 'made', 'smaller', 'ones', 'how', 'smaller', 'units', 'arranged', 'within', 'polymer', 'issue', 'havent', 'addressed', 'closely', 'yet', 'topology', 'study', 'three-dimensional', 'shapes', 'relationships', 'individual', 'parts', 'arranged', 'within', 'whole', 'lets', 'take', 'look', 'topology', 'polymers', 'at', 'simplest', 'level', 'thinking', 'polymers', 'chains', 'monomers', 'strung', 'together', 'like', 'beads', 'string', 'we', 'even', 'use', 'verb', 'enchain', 'describe', 'act', 'taking', 'monomer', 'tying', 'larger', 'polymer', 'the', 'chain', 'basic', 'polymer', 'structures', 'this', 'topology', 'often', 'referred', 'simply', 'chain', 'underscore', 'structure', 'linear', 'polymer', 'even', 'simple', 'chain', 'context', 'additional', 'structural', 'features', 'worth', 'considering', 'these', 'features', 'connected', 'polymer', 'chain', 'grows', 'individual', 'monomers', 'if', 'polymer', 'results', 'chain', 'reaction', 'typically', 'used', 'polymerization', 'alkenes', 'growing', 'chain', 'usually', 'two', 'distinct', 'ends', 'one', 'end', 'sometimes', 'called', 'tail', 'site', 'first', 'monomer', 'incorporated', 'polymer', 'well', 'remnant', 'initiator', 'begin', 'polymerization', 'process', 'the', 'end', 'sometimes', 'called', 'head', 'commonly', 'growing', 'end', 'active', 'site', 'place', 'new', 'monomers', 'enchained', 'polymer', 'thats', 'happens', 'cases', 'initiator', 'capable', 'reacting', 'one', 'monomer', 'starting', 'chain', 'reaction', 'however', 'sometimes', 'initiators', 'difunctional', 'difunctional', 'initiators', 'capable', 'getting', 'two', 'monomers', 'start', 'growing', 'polymer', 'chain', 'one', 'direction', 'in', 'case', 'initiator', 'fragment', 'left', 'behind', 'middle', 'new', 'growing', 'chain', 'grows', 'outward', 'middles', 'both', 'ends', 'chain', 'growing', 'ends', 'in', 'type', 'growth', 'chain', 'referred', 'telechelic.', 'sometimes', 'polymer', 'chains', 'simple', 'linear', 'structures', 'instead', 'chains', 'branch', 'this', 'topology', 'called', 'branched', 'instead', 'looking', 'like', 'snake', 'piece', 'spaghetti', 'structure', 'looks', 'like', 'length', 'seaweed', 'branching', 'sometimes', 'artifact', 'polymer', 'made', 'sometimes', 'monomers', 'may', 'lead', 'linear', 'polymer', 'branched', 'one', 'the', 'prime', 'example', 'polyethylene', 'form', 'high-density', 'polyethylene', '(hdpe)', 'low-density', 'polyethylene', '(ldpe)', 'depending', 'conditions', 'ethylene', 'enchained.', 'in', 'branched', 'polymer', 'smaller', 'chains', 'grow', 'like', 'limbs', 'along', 'tree', 'trunk', 'leaves', 'along', 'stem', 'we', 'take', 'arrangement', 'step', 'something', 'looks', 'like', 'net', 'in', 'call', 'crosslinked', 'structure', 'branches', 'connect', 'one', 'main', 'chain', 'next', 'tying', 'together', 'one', 'big', 'piece', 'sometimes', 'type', 'structure', 'called', 'thermoset', 'the', 'word', 'thermoset', 'really', 'description', 'physical', 'property', 'polymer', 'contrast', 'thermoplastic', 'a', 'thermoplastic', 'polymer', 'melted', 'reformed', 'new', 'shapes', 'polymerization', 'in', 'contrast', 'thermoset', 'polymerized', 'retains', 'shape', 'even', 'heated', 'doesnt', 'melt', 'these', 'terms', 'connotations', 'topology', 'material', 'nonetheless.', 'the', 'reason', 'thermoplastics', 'melted', 'formed', 'new', 'shapes', 'made', 'separate', 'molecules', 'the', 'molecules', 'may', 'long', 'may', 'even', 'branched', 'high', 'enough', 'temperatures', 'molecules', 'move', 'completely', 'independently', 'they', 'melt', 'material', 'comprised', 'separate', 'molecules', 'take', 'new', 'shapes', 'in', 'thermoset', 'crosslinks', 'connect', 'different', 'chains', 'material', 'forming', 'bridges', 'span', 'chain', 'chain', 'chain', 'essentially', 'uniting', 'material', 'one', 'big', 'molecule', 'if', 'one', 'big', 'molecule', 'chains', 'never', 'move', 'completely', 'independently', 'material', 'cant', 'form', 'new', 'shape', 'looked', 'another', 'way', 'crosslinks', 'tie', 'main', 'chains', 'place', 'they', 'may', 'able', 'move', 'around', 'never', 'get', 'far', 'if', 'amount', 'crosslinking', 'sufficient', 'always', 'hold', 'material', 'basic', 'shape', 'of', 'course', 'little', 'bit', 'crosslinking', 'may', 'effect', 'you', 'may', 'two', 'three', 'chains', 'tied', 'together', 'form', 'one', 'big', 'molecule', 'behave', 'like', 'highly', 'branched', 'chains', 'like', 'extended', 'nets', 'its', 'also', 'worth', 'noting', 'term', 'crosslink', 'actually', 'mean', 'different', 'things', 'used', 'different', 'ways', 'sometimes', 'crosslinks', 'refer', 'true', 'covalent', 'bonds', 'connecting', 'two', 'chains', 'together', 'these', 'connections', 'called', 'chemical', 'crosslinks;', 'word', 'chemical', 'refers', 'covalent', 'bonds', 'alternatively', 'chains', 'connected', 'strong', 'intermolecular', 'forces', 'thats', 'thing', 'course', 'intermolecular', 'forces', 'overcome', 'sufficient', 'energy', 'point', 'chains', 'may', 'longer', 'tied', 'together', 'these', 'connections', 'called', 'physical', 'crosslinks', 'distinguish', 'permanent', 'bonds', 'to', 'use', 'biology', 'analogy', 'sulfide', 'bonds', 'proteins', 'would', 'example', 'chemical', 'crosslinks', 'hold', 'protein', 'firmly', 'one', 'shape', 'chemical', 'reaction', 'required', 'break', 'connection', 'the', 'ordinary', 'hydrogen', 'bonds', 'prevalent', 'protein', 'physical', 'crosslinks', 'because', 'overcome', 'adding', 'heat', 'proteins', 'sensitive', 'temperature', 'dendrimers', 'another', 'type', 'branched', 'structure', 'term', 'comes', 'greek', 'dendron', 'tree', 'dendrimers', 'differ', 'regular', 'branched', 'polymers', 'much', 'regular', 'branching', 'pattern', 'a', 'dendrimer', 'actually', 'grow', 'outward', 'center', 'branching', 'regular', 'intervals', 'many', 'dendrimers', 'polyamides', 'although', 'types', 'well', 'generally', 'least', 'one', 'monomers', 'trifunctional', 'introduces', 'branching', 'predictable', 'way', 'as', 'dendrimers', 'grow', 'outward', 'layer', 'layer', 'often', 'described', 'terms', 'generations', 'suppose', 'start', 'trifunctional', 'monomer', 'middle', 'thats', 'generation', '0', 'if', 'monomer', 'polymerized', 'outward', 'another', 'set', 'trifunctional', 'monomers', 'attached', 'edge', 'first', 'generation', 'dendrimer', 'if', 'keep', 'going', 'add', 'another', 'layer', 'trifunctional', 'monomers', 'second', 'generation', 'dendrimer', 'these', 'dendrimers', 'start', 'look', 'circular', 'paper', 'like', 'pancakes', 'steric', 'interactions', 'groups', 'forces', 'things', 'three', 'dimensions', 'as', 'result', 'dendrimers', 'roughly', 'spherical', 'shape', 'in', 'examples', 'polymers', 'seen', 'chain', 'actually', 'composed', 'two', 'different', 'monomers', 'thats', 'true', 'case', 'polyamides', 'nylon-66', 'in', 'example', 'chain', 'composed', 'difunctional', 'amines', 'alternating', 'difunctional', 'carboxyloids', '(such', 'carboxylic', 'acids', 'acid', 'chlorides)', 'we', 'think', 'polymer', 'like', 'composed', 'two', 'different', 'monomers', 'of', 'course', 'complementary', 'reactivity', 'alternate', 'amine', 'carboxyloid', 'form', 'amide', 'we', 'think', 'polymers', 'co-polymers,', 'meaning', 'formed', 'one', 'kind', 'monomer', 'we', 'go', 'say', 'alternating', 'co-polymers', 'two', 'different', 'monomers', 'alternate', 'along', 'chain', 'in', 'cases', 'need', 'two', 'different', 'monomers', 'alternate', 'way', 'nylon-66', 'if', 'take', 'mixture', 'alkenes', 'capable', 'forming', 'polymers', 'polymerize', 'together', 'may', 'well', 'get', 'randomly', 'enchained', 'growing', 'polymer', 'this', 'arrangement', 'called', 'random', 'co-polymer', 'sometimes', 'statistical', 'co-polymer', 'for', 'example', 'maybe', 'manage', 'get', 'random', 'sequence', 'propene', 'vinyl', 'chloride', 'units', 'along', 'polymer.', 'notice', 'example', 'still', 'zig-zag', 'main', 'chain', 'polymer', 'changes', 'one', 'monomer', 'group', 'attached', 'main', 'chain', 'it', 'sometimes', 'useful', 'think', 'main', 'chain', 'backbone', 'separately', 'attached', 'pendant', 'groups', 'in', 'case', 'looks', 'like', 'polyethylene', 'backbone', 'pendant', 'chlorides', 'methyls', 'randomly', 'attached.', 'a', 'random', 'arrangement', 'isnt', 'possibility', 'maybe', 'vinyl', 'chlorides', 'polymerized', 'row', 'propenes', 'incorporated', 'the', 'result', 'would', 'diblock', 'co-polymer', 'solid', 'block', 'poly(vinyl', 'chloride)', 'one', 'end', 'chain', 'solid', 'block', 'polypropylene', 'end', 'there', 'couple', 'ways', 'could', 'happen', 'maybe', 'waited', 'vinyl', 'chlorides', 'enchained', 'adding', 'propene', 'vinyl', 'chlorides', 'enchained', 'tail', 'end', 'propenes', 'added', 'growing', 'end', 'later', 'alternatively', 'maybe', 'added', 'vinyl', 'chlorides', 'underwent', 'polymerization', 'whole', 'lot', 'faster', 'propenes', 'vinyl', 'chlorides', 'became', 'enchained', 'propenes', 'chance', 'those', 'possibilities', 'assume', 'monomers', 'enchained', 'linearly', 'there', 'possibilities', 'maybe', 'one', 'set', 'monomers', 'form', 'polymer', 'backbone', 'set', 'form', 'pendant', 'branches', 'along', 'chain', 'that', 'arrangement', 'called', 'graft', 'co-polymer,', 'though', 'little', 'apple', 'trees', 'grafted', 'onto', 'trunk', 'another', 'breed', 'problem', 'mp11', 'problem', 'mp12', 'problem', 'mp13', 'problem', 'mp21', 'problem', 'mp22', 'problem', 'mp23', 'problem', 'mp31', 'problem', 'mp32', 'problem', 'mp41', 'problem', 'mp42', 'problem', 'mp43', 'problem', 'mp51', 'problem', 'mp52', 'problem', 'mp53', 'problem', 'mp61', 'problem', 'mp71', 'problem', 'mp72', 'molecular', 'weight', 'one', 'central', 'aspects', 'polymer', 'properties', 'of', 'course', 'molecules', 'molecular', 'weights', 'it', 'might', 'seem', 'obvious', 'molecular', 'weight', 'essential', 'property', 'molecular', 'compound', 'in', 'polymers', 'molecular', 'weight', 'takes', 'added', 'significance', 'thats', 'polymer', 'large', 'molecule', 'made', 'repeating', 'units', 'many', 'repeating', 'units', 'thirty', 'a', 'thousand', 'a', 'million', 'any', 'possibilities', 'might', 'still', 'considered', 'representative', 'material', 'molecular', 'weights', 'would', 'different', 'would', 'properties', 'that', 'variation', 'introduces', 'unique', 'aspects', 'polymer', 'molecular', 'weight', 'because', 'polymers', 'assembled', 'smaller', 'molecules', 'length', '(and', 'consequently', 'molecular', 'weight)', 'polymer', 'chain', 'depends', 'number', 'monomers', 'enchained', 'polymer', 'the', 'number', 'enchained', 'monomers', 'average', 'polymer', 'chain', 'material', 'called', 'degree', 'polymerization', '(dp)', 'notice', 'key', 'point', 'average', 'in', 'given', 'material', 'chains', 'added', 'monomers', 'chains', 'added', 'fewer', 'why', 'difference', 'first', 'polymer', 'growth', 'dynamic', 'process', 'it', 'requires', 'monomers', 'come', 'together', 'react', 'what', 'one', 'monomer', 'starts', 'reacting', 'forming', 'growing', 'chain', 'others', 'get', 'started', 'with', 'head', 'start', 'chain', 'become', 'longer', 'rest', 'what', 'something', 'goes', 'wrong', 'one', 'growing', 'chains', 'longer', 'add', 'new', 'monomers', 'that', 'chain', 'experienced', 'early', 'death', 'never', 'grow', 'long', 'others', 'as', 'result', 'speaking', 'molecular', 'weight', 'polymer', 'always', 'talking', 'average', 'value', 'some', 'chains', 'material', 'longer', '(and', 'heavier)', 'chains', 'material', 'shorter', '(and', 'lighter)', 'as', 'group', 'measurements', 'helpful', 'know', 'widely', 'distributed', 'individual', 'values', 'really', 'in', 'polymer', 'chemistry', 'width', 'distribution', 'molecular', 'weights', 'described', 'dispersity', '(ð', 'also', 'called', 'older', 'texts', 'polydispersity', 'polydispersity', 'index', 'pdi)', 'the', 'dispersity', 'polymer', 'sample', 'often', '1', '2', '(although', 'even', 'higher', '2)', 'the', 'closer', '1', 'narrower', 'distribution', 'that', 'dispersity', '10', 'would', 'mean', 'chains', 'sample', 'exactly', 'length', 'molecular', 'weight', 'the', 'original', 'idea', 'dispersity', 'based', 'alternative', 'methods', 'measuring', 'molecular', 'weight', '(or', 'chain', 'length)', 'polymer', 'sample', 'one', 'set', 'methods', 'gave', 'something', 'called', 'number', 'average', 'molecular', 'weight', '(symbol', 'mn)', 'these', 'methods', 'essentially', 'took', 'weight', 'sample', 'counted', 'molecules', 'sample', 'therefore', 'found', 'average', 'weight', 'molecule', 'sample', 'a', 'classic', 'example', 'approach', 'colligative', 'properties', 'experiment', 'freezing', 'point', 'depression', 'you', 'know', 'impurities', 'liquid', 'tend', 'disrupt', 'intermolecular', 'interactions', 'lower', 'freezing', 'point', 'liquid', 'you', 'may', 'also', 'know', 'amount', 'freezing', 'gets', 'lowered', 'depends', 'number', 'molecules', 'ions', 'get', 'dissolved', 'hence', 'weigh', 'sample', 'polymer', 'dissolve', 'solvent', 'measure', 'freezing', 'point', 'could', 'figure', 'number', 'molecules', 'dissolved', 'consequently', 'arrive', 'mn', 'thats', 'easy', 'practice', 'freezing', 'point', 'depressions', 'small', 'theyre', 'used', 'often', 'anymore', 'a', 'common', 'example', 'kind', 'measurement', 'widely', 'used', 'determine', 'mn', 'today', 'end', 'group', 'analysis', 'in', 'end', 'group', 'analysis', 'use', '1h', 'nmr', 'measurements', 'determine', 'ratio', 'specific', 'proton', 'repeat', 'units', 'specific', 'proton', 'end', 'group', 'remember', 'end', 'group', 'might', 'something', 'like', 'initiator', 'added', 'onto', 'first', 'monomer', 'get', 'polymerization', 'going', 'by', 'end', 'polymerization', 'still', 'found', 'end', 'polymer', 'chain', 'end', 'group', 'there', 'one', 'per', 'chain', 'whereas', 'lots', 'monomers', 'enchained', 'polymer', 'ratio', 'enchained', 'monomers', 'end', 'group', 'tells', 'us', 'long', 'chain', 'the', 'set', 'methods', 'upon', 'dispersity', 'based', 'gave', 'something', 'called', 'weight', 'average', 'molecular', 'weight', '(symbol', 'mw)', 'the', 'classic', 'example', 'light-scattering', 'experiment', 'in', 'experiment', 'solution', 'polymer', 'exposed', 'beam', 'light', 'resulting', 'scattered', 'light', '--', 'coming', 'sample', 'different', 'directions', '--', 'analyzed', 'determine', 'size', 'polymer', 'chains', 'solution', 'the', 'results', 'heavily', 'influenced', 'larger', 'molecules', 'solution', 'as', 'result', 'measurement', 'molecular', 'weight', 'always', 'higher', 'measurements', 'based', 'counting', 'every', 'single', 'molecule', 'the', 'resulting', 'ratio', 'ð', 'mw', 'mn', 'became', 'known', 'polydispersity', 'index', 'recently', 'dispersity', 'because', 'mw', 'always', 'strongly', 'influenced', 'longer', 'chains', 'little', 'bigger', 'mn', 'therefore', 'dispersity', 'always', 'bigger', '10', 'nowadays', 'molecular', 'weight', 'dispersity', 'commonly', 'measured', 'using', 'gel', 'permeation', 'chromatography', '(gpc)', 'synonymous', 'size-exclusion', 'chromatography', '(sec)', 'this', 'method', 'high-performance', 'liquid', 'chromatography', '(hplc)', 'technique', 'solvent', 'containing', 'sample', 'polymer', 'pumped', 'specialized', 'chromatography', 'column', 'capable', 'separating', 'molecules', 'based', 'size', 'differences', 'as', 'sample', 'emerges', 'column', 'detected', 'recorded', 'most', 'commonly', 'presence', 'sample', 'solvent', 'emerging', 'column', 'causes', 'slight', 'change', 'refractive', 'index', 'a', 'graph', 'refractive', 'index', 'versus', 'time', 'presents', 'record', 'amount', 'sample', 'emerging', 'column', 'given', 'time', 'because', 'column', 'separated', 'molecules', 'based', 'size', 'time', 'axis', 'corresponds', 'indirectly', 'chain', 'length', 'molecular', 'weight', 'how', 'column', 'separate', 'molecules', 'based', 'size', 'the', 'column', 'packed', 'porous', 'material', 'usually', 'insoluble', 'polymer', 'beads', 'the', 'pore', 'sizes', 'vary', 'these', 'pores', 'crucial', 'separation', 'molecules', 'flowing', 'column', 'may', 'tarry', 'pores', 'smaller', 'molecules', 'could', 'become', 'delayed', 'pores', 'material', 'whereas', 'larger', 'molecules', 'delayed', 'largest', 'pores', 'consequently', 'longer', 'elution', 'time', 'corresponds', 'lower', 'molecular', 'weight', 'if', 'injected', 'series', 'different', 'polymers', 'gpc', 'different', 'molecular', 'weight', 'distribution', 'would', 'observe', 'one', 'eluting', 'different', 'time', 'whats', 'peak', 'may', 'broader', 'narrower', 'depending', 'dispersity', 'particular', 'sample', 'the', 'wider', 'peak', 'gpc', 'broader', 'distribution', 'molecular', 'weights', 'narrower', 'peaks', 'uniform', 'chains', 'normally', 'software', 'package', 'analyzes', 'curve', 'determine', 'dispersity', 'note', 'x', 'axis', 'gpc', 'trace', 'commonly', 'labeled', 'elution', 'time', 'normally', 'runs', 'left', 'right', 'however', 'often', 'x', 'axis', 'labeled', 'molecular', 'wright', 'really', 'quantity', 'interested', 'in', 'fact', 'sometimes', 'axis', 'reversed', 'peaks', 'higher', 'molecular', 'weights', 'appear', 'right', 'feel', 'natural', 'look', 'way', 'you', 'need', 'look', 'carefully', 'data', 'see', 'displayed', 'there', 'problems', 'relying', 'gpc', 'molecular', 'weight', 'measurements', 'the', 'main', 'difficulty', 'polymers', 'solution', 'tend', 'coil', 'balls', 'coils', 'contain', 'greater', 'lesser', 'amounts', 'solvent', 'depending', 'strongly', 'polymer', 'solvent', 'interact', 'if', 'interacts', 'strongly', 'solvent', 'pull', 'lots', 'solvent', 'molecules', 'inside', 'coils', 'the', 'coil', 'get', 'bigger', 'make', 'room', 'internal', 'solvent', 'molecules', 'if', 'doesnt', 'interact', 'strongly', 'solvent', 'mostly', 'stick', 'blocking', 'solvent', 'molecules', 'there', 'broad', 'range', 'behaviors', 'as', 'result', 'different', 'polymers', 'may', 'swell', 'different', 'extents', 'different', 'solvents', 'that', 'matters', 'gpc', 'really', 'using', 'size', 'polymer', 'coil', 'index', 'molecular', 'weight', 'comparing', 'gpc', 'traces', 'two', 'different', 'kinds', 'polymers', 'done', 'caution', 'problem', 'cp11', 'in', 'following', 'cases', 'state', 'polymer', 'higher', 'molecular', 'weight', 'one', 'narrower', 'dispersity', 'problem', 'cp12', 'calculate', 'molecular', 'weight', 'following', 'samples', 'problem', 'cp13', 'use', 'nmr', 'end', 'group', 'analysis', 'determine', 'degrees', 'polymerization', 'following', 'samples', 'polymers', 'come', 'occupy', 'important', 'niche', 'materials', 'use', 'every', 'day', 'what', 'makes', 'unique', 'why', 'properties', 'arent', 'easily', 'replicated', 'materials', 'lets', 'start', 'property', 'probably', 'familiar', 'viscosity', 'term', 'use', 'describe', 'thickness', 'different', 'liquids', 'for', 'example', 'say', 'honey', 'viscous', 'water', 'motor', 'oil', 'viscous', 'gasoline', 'when', 'say', 'mean', 'water', 'much', 'easier', 'stir', 'pour', 'honey', 'the', 'honey', 'moves', 'slowly', 'it', 'resists', 'movement', 'spoon', 'stir', 'viscosity', 'often', 'described', 'general', 'terms', 'resistance', 'flow', 'the', 'honey', 'doesnt', 'flow', 'easily', 'especially', 'compared', 'something', 'like', 'water.', 'now', 'honey', 'isnt', 'polymer', 'its', 'concentrated', 'solution', 'it', 'contains', 'little', 'bit', 'water', 'whole', 'lot', 'sugars', 'plus', 'small', 'molecules', 'produced', 'plants', 'bees', 'gathered', 'nectar', 'make', 'honey', 'the', 'sugars', 'arent', 'polymers', 'either', 'simple', 'monosaccharides', 'glucose', 'fructose', 'nevertheless', 'high', 'viscosity', 'something', 'honey', 'common', 'polymer', 'solutions', 'well', 'oligomers', 'short-chain', 'polymers', 'liquids', 'instead', 'solids', 'why', 'honey', 'viscous', 'partly', 'sugar', 'molecules', 'water', 'much', 'larger', 'water', 'molecules', 'experience', 'lot', 'drag', 'move', 'solution', 'compared', 'plain', 'water', 'the', 'strong', 'intermolecular', 'attractions', 'individual', 'sugar', 'molecules', 'also', 'major', 'factor', 'sugars', 'covered', 'oh', 'hydroxyl', 'groups', 'capable', 'strong', 'hydrogen', 'bonds', 'as', 'molecules', 'move', 'past', 'concentrated', 'solution', 'cling', 'slowing', 'flow', 'liquid', 'the', 'motor', 'oil', 'example', 'works', 'similar', 'way', 'motor', 'oil', 'gasoline', 'composed', 'class', 'compounds', 'hydrocarbons', 'composed', 'carbon', 'chains', 'covered', 'hydrogen', 'atoms', 'theyre', 'similar', 'the', 'intermolecular', 'attractions', 'molecules', 'much', 'weaker', 'sugar', 'molecules', 'theyre', 'london', 'dispersion', 'forces', 'the', 'major', 'difference', 'motor', 'oil', 'gasoline', 'motor', 'oil', 'contains', 'much', 'longer', 'hydrocarbon', 'chains', 'the', 'molecules', 'bigger', 'experience', 'drag', 'move', 'liquid', 'the', 'amount', 'intermolecular', 'interactions', 'also', 'crucial', 'determining', 'tightly', 'two', 'molecules', 'hold', 'onto', 'factor', 'especially', 'important', 'weak', 'london', 'dispersion', 'forces', 'small', 'advantage', 'goes', 'long', 'way', 'those', 'longer', 'hydrocarbon', 'chains', 'motor', 'oil', 'cling', 'much', 'strongly', 'shorter', 'ones', 'gasoline', 'gasoline', 'flows', 'much', 'easily', 'motor', 'oil', 'for', 'similar', 'reasons', 'oligomers', 'polymers', 'liquids', 'also', 'display', 'high', 'viscosity', 'the', 'extended', 'contact', 'long', 'molecular', 'chains', 'leads', 'enhanced', 'intermolecular', 'attractions', 'contribute', 'resistance', 'flow', 'increased', 'drag', 'also', 'factor', 'even', 'solution', 'large', 'molecules', 'encounter', 'resistance', 'move', 'past', 'solvent', 'molecules', 'compared', 'resistance', 'smaller', 'molecules', 'would', 'experience', 'in', 'fact', 'viscosity', 'measurements', 'polymer', 'solutions', 'another', 'way', 'determine', 'size', 'polymer', '--', 'leading', 'chain', 'length', 'molecular', 'weight', 'the', 'larger', 'polymer', 'drag', 'also', 'intermolecular', 'attraction', 'higher', 'viscosity', 'so', 'viscosity', 'increases', 'molecular', 'weight', 'necessarily', 'linear', 'way', 'that', 'make', 'difficult', 'use', 'graph', 'like', 'predict', 'molecular', 'weight', 'viscosity', 'measurement', 'one', 'common', 'thing', 'situation', 'take', 'log', 'values', 'that', 'approach', 'often', 'give', 'straight', 'line', 'two', 'things', 'related', 'often', 'nature', 'one', 'physical', 'property', 'depends', 'another', 'follow', 'power', 'law', 'that', 'means', 'one', 'quantity', 'depends', 'raised', 'exponent.', 'in', 'case', 'relationship', 'described', 'mark-houwink', 'equation', '[η', 'kmα]', 'in', 'equation', 'η', 'intrinsic', 'viscosity', 'thats', 'viscosity', 'attributed', 'solute', 'rather', 'viscosity', 'solvent', 'm', 'molecular', 'weight', 'k', 'α', 'constants', 'particular', 'polymer', 'if', 'take', 'log', 'sides', 'equation', '--', 'thats', 'ok', 'mathematically', 'like', 'could', 'multiply', 'sides', 'equation', 'four', 'subtract', 'three', 'sides', 'still', 'equivalent', 'expression', '--', 'get', 'log[η', 'log(kma)]', 'if', 'take', 'log', 'two', 'things', 'multiplied', 'together', 'adding', 'individual', 'logs', 'things', 'log[η', 'log', 'k', 'log(ma)]', 'and', 'take', 'log', 'something', 'raised', 'exponent', 'exponent', 'comes', 'become', 'coefficient', 'log[η', 'log', 'k', 'αlog', 'm]', 'log[η', 'αlogm', 'log', 'k]', 'that', 'looks', 'like', 'equation', 'straight', 'line', 'mx', 'b', 'here', 'plot', 'logη', 'axis', 'logm', 'x', 'axis', 'get', 'straight', 'line', 'constants', 'α', 'k', 'slope', 'y-intercept', 'respectively', 'this', 'approach', 'yield', 'absolute', 'measurement', 'you', 'couldnt', 'take', 'two', 'completely', 'different', 'polymers', 'take', 'viscosity', 'measurements', 'make', 'conclusions', 'one', 'higher', 'average', 'molecular', 'weight', 'however', 'took', 'series', 'polymers', 'kind', 'measured', 'viscosities', 'solution', 'would', 'able', 'deduce', 'ones', 'longer', 'ones', 'shorter', 'in', 'fact', 'dependence', 'viscosity', 'molecular', 'weight', 'polymers', 'complicated', 'graph', 'suggests', 'if', 'deal', 'large', 'enough', 'polymers', 'graph', 'starts', 'look', 'like', 'one', 'why', 'bend', 'remember', 'polymers', 'solution', 'coiled', 'balls', 'a', 'longer', 'chain', 'corresponds', 'bigger', 'coil', 'at', 'point', 'chains', 'become', 'big', 'enough', 'arent', 'likely', 'stay', 'coiled', 'independently', 'they', 'become', 'likely', 'interact', 'they', 'become', 'entangled', 'there', 'plenty', 'factors', 'would', 'take', 'account', 'using', 'viscosity', 'assess', 'molecular', 'weight', 'one', 'concentration', 'example', 'the', 'concentrated', 'polymer', 'solution', 'large', 'molecules', 'exerting', 'drag', 'interacting', 'higher', 'concentration', 'leads', 'higher', 'viscosity', 'measurement', 'consequently', 'would', 'make', 'sure', 'accounted', 'polymer', 'concentrations', 'used', 'method', 'problem', 'pp21', 'estimate', 'values', 'α', 'k', 'polymer', 'provided', 'plot', 'problem', 'pp22', 'suppose', 'polymer', 'described', 'plot', 'if', 'sample', 'molecular', 'weight', '1', 'million', 'gmol', 'would', 'expect', 'intrinsic', 'viscosity', 'conditions', 'experiment', 'problem', 'pp23', 'suppose', 'calibration', 'plot', 'particular', 'polymer', 'if', 'measure', 'intrinsic', 'viscosity', 'sample', 'find', 'value', '800', 'ml', 'g', 'molecular', 'weight', 'sample', 'problem', 'pp24', 'ethanol', '(ch3ch2oh)', 'absolute', 'viscosity', '1095', 'centipoise', 'whereas', 'ethylene', 'glycol', '(hoch2ch2oh)', 'absolute', 'viscosity', '162', 'centipoise', 'explain', 'two', 'values', 'different', 'viscosity', 'measurements', 'realm', 'field', 'science', 'called', 'rheology', 'rheology', 'literally', 'study', 'flow', 'another', 'simple', 'definition', 'attributed', 'chemical', 'engineer', 'chris', 'macosko', 'university', 'minnesota', 'study', 'happens', 'squish', 'stuff', 'theres', 'element', 'force', 'pressure', 'comes', 'play', 'too.', 'one', 'common', 'ways', 'assessing', 'properties', 'rheology', 'place', 'sample', 'two', 'parallel', 'plates', 'move', 'one', 'plate', 'respect', 'one', 'plate', 'says', 'still', 'one', 'moves', 'what', 'happens', 'liquid', 'plates', 'there', 'ought', 'friction', 'stationary', 'plate', 'liquid', 'keep', 'liquid', 'still', 'there', 'also', 'ought', 'friction', 'moving', 'plate', 'liquid', 'make', 'liquid', 'move', 'along', 'speed', 'plate', 'so', 'one', 'extreme', 'liquid', 'moving', 'along', 'sliding', 'plate', 'extreme', 'liquid', 'perfectly', 'still', 'if', 'imagine', 'liquid', 'two', 'extremes', 'divided', 'thin', 'layers', 'layer', 'must', 'moving', 'slightly', 'different', 'speed', 'next', 'in', 'diagram', 'symbol', 'u', 'stands', 'speed', 'layer', 'liquid', 'the', 'arrow', 'beside', 'layer', 'meant', 'convey', 'relative', 'speed', 'top', 'layer', 'moving', 'fastest', 'next', 'layer', 'little', 'slower', 'bottom', 'layer', 'isnt', 'moving', 'there', 'important', 'quantity', 'called', 'strain', 'rate', 'shear', 'rate', '(given', 'symbol', 'greek', 'letter', 'gamma', 'γ', 'dot', 'top)', 'describes', 'speed', 'liquid', 'moving', 'along', 'x-axis', '(left', 'right)', 'changes', 'layer', 'layer', 'y-direction', 'what', 'picture', 'viscosity', 'well', 'order', 'get', 'top', 'plate', 'move', 'apply', 'sort', 'force', 'in', 'rheology', 'force', 'given', 'per', 'unit', 'area', '(like', 'pressure)', 'force', 'per', 'unit', 'area', 'described', 'stress', '(given', 'symbol', 'greek', 'letter', 'sigma', 'σ)', 'what', 'wanted', 'slide', 'plate', 'even', 'faster', 'what', 'would', 'happen', 'liquid', 'well', 'shear', 'rate', 'would', 'go', 'the', 'top', 'layer', 'would', 'moving', 'even', 'faster', 'bottom', 'layer', 'would', 'still', 'stationary', 'and', 'would', 'get', 'faster', 'shear', 'rate', 'we', 'would', 'push', 'top', 'plate', 'little', 'harder', 'the', 'result', 'graph', 'looks', 'like', 'the', 'graph', 'says', 'shear', 'stress', 'shear', 'rate', 'directly', 'related', 'the', 'stress', 'called', 'shear', 'stress', 'direction', 'force', 'parallel', 'liquid', 'causing', 'shear', 'strain', 'when', 'increase', 'shear', 'rate', 'shear', 'stress', 'also', 'increases', 'proportionally', 'so', 'experiment', 'using', 'water', 'liquid', 'plates', 'would', 'get', 'linear', 'relationship', 'shear', 'stress', 'shear', 'rate', 'thats', 'sir', 'isaac', 'newton', 'described', 'behavior', 'liquids', 'see', 'linear', 'relationship', 'stress', 'strain', 'describe', 'liquid', 'newtonian', 'liquid', 'what', 'happens', 'experiment', 'honey', 'instead', 'water', 'what', 'would', 'happen', 'example', 'exerted', 'certain', 'stress', 'get', 'certain', 'shear', 'rate', 'water', 'wanted', 'get', 'shear', 'rate', 'using', 'honey', 'well', 'honey', 'thicker', 'water', 'its', 'going', 'move', 'easily', 'we', 'apply', 'greater', 'stress', 'order', 'achieve', 'shear', 'rate', 'measured', 'water', 'that', 'would', 'true', 'every', 'possible', 'shear', 'rate', 'if', 'graph', 'shear', 'stress', 'vs', 'shear', 'strain', 'honey', 'would', 'get', 'linear', 'relationship', 'like', 'water', 'value', 'shear', 'stress', 'would', 'always', 'higher', 'water', 'the', 'slope', 'line', 'honey', 'would', 'greater', 'slope', 'line', 'water', 'viscosity', 'formally', 'defined', 'slope', 'line', 'honey', 'thicker', 'water', 'requires', 'higher', 'higher', 'stresses', 'achieve', 'strain', 'rates', 'would', 'get', 'using', 'lower', 'stresses', 'water', 'in', 'simple', 'kitchen', 'analog', 'experiment', 'push', 'spoon', 'harder', 'stir', 'honey', 'stirring', 'water', 'honey', 'water', 'newtonian', 'liquids', 'the', 'relationship', 'simple', 'increase', 'shear', 'stress', 'increase', 'shear', 'rate', 'not', 'liquids', 'behave', 'way', 'polymer', 'solutions', 'dont', 'example', 'they', 'display', 'non-linear', 'relationship', 'shear', 'stresses', 'needed', 'get', 'higher', 'shear', 'rates', 'high', 'would', 'expect', 'that', 'behavior', 'described', 'shear', 'thinning', 'what', 'makes', 'polymer', 'solutions', 'remember', 'solution', 'individual', 'polymer', 'molecule', 'tends', 'coil', 'ball', 'normally', 'balls', 'spherical', 'at', 'high', 'shear', 'rates', 'distort', 'ellipses', 'that', 'reduces', 'amount', 'drag', 'making', 'solution', 'little', 'less', 'viscous', 'would', 'expect', 'the', 'shear', 'stress', 'still', 'increases', 'higher', 'shear', 'rates', 'much', 'newtonian', 'fluid', 'thats', 'remember', 'polymers', 'display', 'chain', 'entanglement', 'especially', 'large', 'chain', 'entanglement', 'increases', 'viscosity', 'solution', 'these', 'chains', 'mobile', 'polymer', 'constantly', 'undergoing', 'conformational', 'changes', 'as', 'solution', 'gets', 'sheared', 'polymer', 'chains', 'may', 'wiggle', 'loose', 'neighbors', 'random', 'conformational', 'changes', 'always', 'result', 'new', 'entanglements', 'right', 'away', 'but', 'happens', 'high', 'rate', 'shear', 'strain', 'the', 'polymer', 'chains', 'still', 'become', 'untangled', 'dont', 'time', 'form', 'new', 'entanglements', 'everything', 'whizzing', 'quickly', 'without', 'entanglements', 'viscosity', 'doesnt', 'increase', 'sharply', 'would', 'expect', 'some', 'liquids', 'behave', 'like', 'water', 'polymer', 'solutions', 'thats', 'consequence', 'enormously', 'long', 'chain', 'structures', 'polymers', 'its', 'worth', 'pointing', 'things', 'dont', 'behave', 'normal', 'way', 'colloids', 'often', 'display', 'shear', 'thickening', 'you', 'see', 'examples', 'videos', 'people', 'running', 'across', 'wading', 'pools', 'filled', 'corn', 'starch', 'water', 'colloidal', 'mixture', 'solid', 'cornstarch', 'particles', 'suspended', 'water', 'dissolved', 'if', 'walk', 'across', 'pool', 'sink', 'if', 'run', 'stay', 'top', 'thats', 'shear', 'thickening', 'solid', 'particles', 'move', 'shearing', 'when', 'walk', 'slide', 'past', 'run', 'collide', 'come', 'stop', 'a', 'bingham', 'plastic', 'hand', 'actually', 'appears', 'solid', 'give', 'good', 'shove', 'flows', 'like', 'liquid', 'peanut', 'butter', 'behaves', 'sort', 'like', 'problem', 'pp31', 'honey', 'newtonian', 'fluid', 'molasses', 'undergoes', 'shear', 'thinning', 'what', 'might', 'deduce', 'composition', 'molasses', 'honey', 'the', 'glass', 'transition', 'probably', 'commonly-cited', 'characteristic', 'polymeric', 'material', 'at', 'glass', 'transition', 'temperature', 'physical', 'nature', 'material', 'changes', 'subtly', 'it', 'goes', 'rubbery', 'flexible', 'material', 'higher', 'temperature', 'glass', 'transition', 'glassy', 'harder', 'material', 'lower', 'temperature', 'the', 'material', 'still', 'solid', 'either', 'way', 'definite', 'change', 'responds', 'stimuli', 'a', 'material', 'could', 'certainly', 'useful', 'either', 'state', 'we', 'might', 'want', 'plastic', 'rigid', 'like', 'water', 'bottle', 'rod', 'shower', 'curtain', 'alternatively', 'might', 'want', 'softer', 'like', 'seat', 'cushion', 'either', 'way', 'might', 'helpful', 'know', 'temperature', 'material', 'change', 'one', 'type', 'the', 'glass', 'transition', 'little', 'like', 'happens', 'gummy', 'bears', 'put', 'ice', 'cream', 'straight', 'bag', 'gummy', 'bears', 'chewy', 'dont', 'feel', 'like', 'break', 'teeth', 'put', 'ice', 'cream', 'changes', 'they', 'become', 'much', 'harder', 'chew', 'the', 'classic', 'explanation', 'glass', 'transition', 'based', 'idea', 'chain', 'flow', 'polymers', 'long-chain', 'molecules', 'given', 'little', 'energy', 'chains', 'move', 'around', 'they', 'wiggle', 'they', 'undergo', 'bond', 'rotations', 'switching', 'one', 'conformation', 'another', 'a', 'block', 'material', 'contains', 'piles', 'chains', 'like', 'nest', 'snakes', 'chain', 'ends', 'loops', 'squirm', 'past', 'continuously', 'chain', 'flow', 'allows', 'material', 'adapt', 'forces', 'exerted', 'we', 'bend', 'eraser', 'chains', 'rubber', 'slide', 'adopt', 'new', 'shape', 'at', 'least', 'thats', 'happens', 'room', 'temperature', 'it', 'might', 'work', 'eraser', 'accidentally', 'dropped', 'liquid', 'nitrogen', 'cold', 'as', 'block', 'material', 'cools', 'contracts', 'the', 'molecules', 'become', 'packed', 'closely', 'together', 'at', 'point', 'free', 'volume', '--', 'thats', 'amount', 'volume', 'material', 'actually', 'taken', 'molecules', '--', 'becomes', 'small', 'allow', 'chains', 'move', 'past', 'all', 'chains', 'need', 'little', 'room', 'around', 'going', 'undergo', 'conformational', 'changes', 'conformational', 'changes', 'polymer', 'chains', 'move', 'without', 'extra', 'room', 'material', 'suddenly', 'becomes', 'less', 'flexible', 'if', 'start', 'glassy', 'state', 'increase', 'temperature', 'volume', 'material', 'expanding', 'free', 'volume', 'increasing', 'at', 'point', 'free', 'volume', 'becomes', 'great', 'enough', 'chains', 'slip', 'past', 'the', 'material', 'becomes', 'flexible', 'it', 'becomes', 'rubbery', 'now', 'transition', 'thing', 'melting', 'the', 'chains', 'completely', 'overcoming', 'interactions', 'gaining', 'freedom', 'movement', 'direction', 'the', 'chains', 'still', 'highly', 'entangled', 'portions', 'chain', 'sliding', 'past', 'overall', 'things', 'changed', 'drastically', 'the', 'block', 'polymer', 'turn', 'gooey', 'puddle', 'liquid', 'well', 'material', 'change', 'glassy', 'rubbery', 'certain', 'temperature', 'consequences', 'material', 'behaves', 'for', 'example', 'tires', 'car', 'meant', 'rubbery', 'flexible', 'factor', 'helps', 'grip', 'road', 'if', 'weather', 'gets', 'cold', 'tires', 'become', 'glassy', 'tires', 'longer', 'amount', 'flexibility', 'dont', 'much', 'traction', 'all-weather', 'tires', 'composed', 'rubber', 'low', 'glass', 'transition', 'temperature', 'helping', 'avoid', 'problem', '(snow', 'tires', 'different', 'thing', 'patterns', 'treads', 'help', 'channel', 'away', 'snow', 'order', 'improve', 'traction)', 'knowing', 'change', 'occur', 'would', 'useful', 'so', 'determine', 'glass', 'transition', 'phase', 'changes', 'melting', 'points', 'measured', 'using', 'calorimetry', 'lets', 'start', 'when', 'solid', 'heated', 'temperature', 'increases', 'that', 'seems', 'simple', 'enough', 'temperature', 'basically', 'measure', 'heat', 'content', 'heat', 'flows', 'measured', 'temperature', 'rises', 'this', 'simple', 'relationship', 'breaks', 'melting', 'point', 'at', 'point', 'heat', 'flowing', 'material', 'consumed', 'breaking', 'intermolecular', 'forces', 'overcoming', 'attractions', 'costs', 'extra', 'energy', 'consequently', 'point', 'temperature', 'rise', 'material', 'stalls', 'temporarily', 'melts', 'this', 'extra', 'heat', 'needed', 'melt', 'material', 'called', 'heat', 'melting', 'commonly', 'heat', 'fusion', 'heat', 'fusion', 'actually', 'refers', 'opposite', 'process', 'material', 'cooled', 'frozen', 'as', 'heat', 'carried', 'away', 'cooling', 'material', 'cools', 'gradually', 'comes', 'point', 'strong', 'intermolecular', 'interactions', 'form', 'giving', 'extra', 'heat', 'the', 'fusion', 'temperature', 'melting', 'temperature', 'heat', 'fusion', 'heat', 'melting', 'one', 'case', 'heat', 'added', 'another', 'case', 'heat', 'given', 'in', 'principle', 'heat', 'something', 'look', 'stalling', 'point', 'temperature', 'find', 'temperature', 'phase', 'transition', 'differential', 'scanning', 'calorimetry', '(dsc)', 'technique', 'commonly', 'used', 'measure', 'phase', 'transitions', 'including', 'glass', 'transition', 'temperature', 'it', 'based', 'ideas', 'experiment', 'run', 'slightly', 'different', 'way', 'in', 'dsc', 'two', 'tiny', 'sample', 'chambers', 'side', 'side', 'one', 'contains', 'material', 'interested', '(empty)', 'one', 'used', 'reference', 'the', 'instrument', 'heats', 'samples', 'constant', 'rate', 'maintaining', 'temperature', 'consequently', 'may', 'actually', 'add', 'heat', 'one', 'chamber', 'reach', '305', '°c', 'time', '306', '°c', 'once', 'reach', 'melting', 'point', 'sample', 'interest', 'heat', 'flow', 'sample', 'must', 'increased', 'keep', 'reference', 'past', 'point', 'heat', 'flow', 'drops', 'back', 'normal', 'level', 'if', 'look', 'dsc', 'scan', 'see', 'graph', 'heat', 'flow', 'y-axis', 'temperature', 'x-axis', 'the', 'heat', 'flow', 'usually', 'stays', 'pretty', 'constant', 'temperature', 'increases', 'at', 'melting', 'point', 'heat', 'flow', 'increases', 'drops', 'back', 'melting', 'point', 'overcome', 'it', 'may', 'drop', 'back', 'level', 'however', 'heat', 'flow', 'recorded', 'related', 'heat', 'capacity', 'material', 'the', 'solid', 'liquid', 'usually', 'respond', 'differently', 'heat', 'in', 'general', 'molecules', 'liquid', 'freely', 'rotate', 'something', 'else', 'added', 'heat', 'liquids', 'thus', 'slightly', 'higher', 'heat', 'capacity', 'corresponding', 'solids', 'all', 'explanation', 'hopefully', 'prepares', 'us', 'dsc', 'study', 'glass', 'transition', 'temperature', 'it', 'works', 'way', 'point', 'if', 'look', 'dsc', 'scan', 'still', 'see', 'heat', 'flow', 'y-axis', 'temperature', 'x-axis', 'at', 'glass', 'transition', 'heat', 'flow', 'bumps', 'slightly', '--', 'remains', 'it', 'looks', 'different', 'melting', 'point', 'a', 'dsc', 'scan', 'melting', 'point', 'shows', 'peak', 'transition', 'temperature', 'a', 'dsc', 'scan', 'glass', 'transition', 'point', 'shows', 'little', 'step', 'like', 'riverbank', 'why', 'different', 'well', 'glass', 'transition', 'really', 'phase', 'change', 'like', 'melting', 'point', 'boiling', 'point', 'it', 'involve', 'physical', 'change', 'state', 'it', 'solid', 'solid', 'afterward', 'no', 'intermolecular', 'interactions', 'must', 'suddenly', 'overcome', 'free', 'molecules', 'instead', 'volume', 'change', 'the', 'free', 'volume', 'became', 'great', 'enough', 'chains', 'slip', 'past', 'chains', 'still', 'clinging', 'together', 'solid-state', 'now', 'additional', 'movement', 'consequences', 'the', 'material', 'becomes', 'flexible', 'if', 'heat', 'flows', 'freedom', 'motion', 'heat', 'distributed', 'in', 'words', 'slight', 'increase', 'heat', 'capacity', 'thats', 'observe', 'dsc', 'one', 'practical', 'note', 'dsc', 'scans', 'actually', 'displayed', 'two', 'different', 'ways', 'the', 'y-axis', 'either', 'display', 'heat', 'flowing', 'heat', 'flowing', 'that', 'means', 'melting', 'points', 'look', 'like', 'peaks', 'like', 'valleys', 'depending', 'data', 'displayed', 'often', 'data', 'labeled', 'arrow', 'says', 'endo', 'tell', 'direction', 'along', 'y-axis', 'means', 'heat', 'flowing', '(or', 'possibly', 'exo,', 'meaning', 'direction', 'means', 'heat', 'flowing', 'out)', 'you', 'need', 'look', 'carefully', 'things', 'seem', 'backward', 'problem', 'cp41', 'for', 'dsc', 'trace', 'state', 'sort', 'transition', 'os', 'occurring', 'temperature', 'why', 'glass', 'transition', 'temperature', 'vary', 'one', 'kind', 'polymer', 'another', 'what', 'structural', 'factors', 'influence', 'glass', 'transition', 'temperature', 'this', 'structure-property', 'relationship', 'isnt', 'straightforward', 'one', 'seem', 'number', 'different', 'variables', 'involved', 'however', 'simplest', 'factors', 'molecular', 'weight', 'the', 'higher', 'molecular', 'weight', 'polymer', 'higher', 'glass', 'transition', 'temperature', 'this', 'relationship', 'true', 'certain', 'point', 'however', 'the', 'non-linear', 'dependence', 'glass', 'transition', 'temperature', 'molecular', 'weight', 'described', 'flory-fox', 'equation', 'tg', 'tg(∞)', '-', 'k', 'mn', 'here', 'tg(∞)', 'refers', 'glass', 'transition', 'temperature', 'infinitely', 'long', 'chain', 'polymer', 'k', 'constant', 'particular', 'polymer', 'polystyrene', 'polyethylene', 'a', 'plot', 'tg', 'vs', 'mn', 'resembles', 'saturation', 'curve', 'line', 'rises', 'sharply', 'gradually', 'stalling', 'continuing', 'parallel', 'x-axis', 'in', 'words', 'although', 'relationship', 'increasing', 'glass', 'transition', 'temperature', 'increasing', 'molecular', 'weight', 'holds', 'true', 'relatively', 'low', 'molecular', 'weights', 'glass', 'transition', 'temperature', 'remains', 'constant', 'threshold', 'molecular', 'weight', 'reached', 'we', 'often', 'think', 'polymers', 'form', 'plastics', 'solid', 'materials', 'serve', 'structural', 'function', 'like', 'water', 'bottle', 'venetian', 'blinds', 'polymers', 'different', 'many', 'solids', 'metals', 'instance', 'crystalline', 'structures', 'atoms', 'form', 'regularly', 'repeating', 'pattern', 'row', 'upon', 'row', 'polymers', 'contrast', 'generally', 'somewhat', 'amorphous', 'think', 'strainer', 'filled', 'cooked', 'spaghetti', 'long', 'chains', 'pasta', 'looping', 'randomly', 'thats', 'molecules', 'amorphous', 'shapeless', 'polymers', 'tend', 'distribute', 'polymers', 'capable', 'forming', 'ordered', 'structures', 'if', 'aligned', 'instead', 'long', 'chains', 'parallel', 'would', 'able', 'get', 'much', 'closer', 'together', 'intermolecular', 'attractions', 'chains', 'would', 'much', 'stronger', 'the', 'energy', 'system', 'would', 'decrease', 'crystalline', 'structure', 'inherently', 'favored', 'so', 'doesnt', 'always', 'happen', 'think', 'polymer', 'cooling', 'completely', 'melted', 'state', 'case', 'structure', 'certainly', 'amorphous', 'as', 'cools', 'material', 'becomes', 'less', 'like', 'liquid', 'less', 'soft', 'with', 'small', 'molecules', 'transition', 'relatively', 'abrupt', 'molecules', 'slide', 'place', 'guided', 'strengthening', 'intermolecular', 'attractions', 'the', 'molecules', 'rotate', 'little', 'face', 'correct', 'direction', 'maybe', 'budge', 'little', 'side', 'with', 'polymers', 'transition', 'much', 'gradual', 'great', 'long', 'chains', 'slide', 'uncoil', 'order', 'lie', 'parallel', 'at', 'point', 'isnt', 'enough', 'energy', 'keep', 'working', 'way', 'optimal', 'alignment', 'as', 'result', 'many', 'polymers', 'semi-crystalline', 'regions', 'called', 'lamellae', 'portions', 'chains', 'aligned', 'parallel', 'also', 'large', 'amorphous', 'areas', 'much', 'randomly', 'oriented', 'as', 'result', 'polymer', 'sample', 'might', '80', 'amorphous', '20', 'chain', 'lengths', 'aligned', 'crystalline', 'lamellae', 'even', 'crystalline', 'domains', 'exert', 'strong', 'influence', 'properties', 'polymer', 'because', 'stronger', 'intermolecular', 'attractions', 'closer', 'chains', 'chains', 'much', 'less', 'able', 'slide', 'past', 'the', 'result', 'material', 'stronger', 'rigid', 'important', 'reliable', 'structural', 'materials', 'how', 'know', 'whether', 'polymer', 'sample', 'contains', 'crystalline', 'domains', 'we', 'use', 'differential', 'scanning', 'calorimetry', 'observe', 'transition', 'ordered', 'crystalline', 'phase', 'disordered', 'melt', 'phase', 'added', 'heat', 'as', 'saw', 'looking', 'glass', 'transition', 'melting', 'point', 'shows', 'peak', 'dsc', 'trace', 'you', 'look', 'temperature', 'tg', 'recall', 'looks', 'like', 'step', 'baseline', 'below', 'tg', 'chains', 'mobile', 'enough', 'move', 'alignment', 'in', 'example', 'see', 'tg', '128', '°c', 'well', 'melting', 'point', 'tm', 'attractions', 'chains', 'lamellae', 'overcome', '155-160', '°c', 'if', 'regular', 'molecular', 'elemental', 'solid', 'melting', 'point', 'would', 'fusion', 'point', 'material', 'would', 'melt', 'temperature', 'warming', 'froze', 'cooling', 'in', 'polymers', 'usually', 'doesnt', 'work', 'way', 'hysteresis', 'observed', 'in', 'hysteresis', 'sample', 'changed', 'previous', 'events', 'leading', 'results', 'dont', 'seem', 'reproducible', 'way', 'would', 'normally', 'expect', 'in', 'general', 'crystallization', 'temperature', 'tc', 'lower', 'melting', 'temperature', 'tm', 'thats', 'material', 'melts', 'chains', 'move', 'alignment', 'chain', 'entanglement', 'hindered', 'getting', 'back', 'alignment', 'leading', 'delay', 'crystallization', 'example', '(pageindex1\\)', 'describe', 'observed', 'following', 'dsc', 'traces', 'crystallinity', 'also', 'probed', 'ways', 'the', 'classic', 'approach', 'x-ray', 'diffraction', 'when', 'x-rays', 'pass', 'ordered', 'materials', 'give', 'rise', 'diffraction', 'patterns', 'bright', 'spots', 'space', 'ricocheting', 'x-rays', 'constructively', 'interfered', 'shining', 'starkly', 'amid', 'blackness', 'x-rays', 'undergone', 'destructive', 'interference', 'the', 'drawing', 'cartoon', 'x-ray', 'diffraction', 'pattern', 'it', 'might', 'captured', 'digital', 'camera', 'earlier', 'days', 'sheet', 'photographic', 'film', 'the', 'bright', 'spots', 'result', 'x-rays', 'scattering', 'middle', 'encountered', 'sample', 'in', 'middle', 'black', 'circle', 'blocks', 'original', 'x-ray', 'beam', 'would', 'otherwise', 'bright', 'allow', 'observation', 'spots', 'around', 'the', 'data', 'x-ray', 'diffraction', 'experiment', 'fed', 'program', 'mathematically', 'deconvolute', 'spots', 'showed', 'the', 'process', 'compared', 'throwing', 'stone', 'shoulder', 'pond', 'watching', 'ripples', 'decide', 'stone', 'must', 'with', 'one', 'stone', 'pretty', 'easy', 'you', 'would', 'find', 'stone', 'center', 'circle', 'ripples', 'with', 'two', 'stones', 'interference', 'patterns', 'make', 'ripples', 'complex', 'may', 'take', 'work', 'determine', 'stone', 'lies', 'pond', 'the', 'picture', 'illustrates', 'results', 'relatively', 'simple', 'x-ray', 'diffraction', 'experiment', 'commonly', 'used', 'crystalline', 'polymer', 'samples', 'the', 'technique', 'called', 'wide-angle', 'x-ray', 'scattering', 'waxs', 'it', 'simply', 'reports', 'angle', 'scattering', 'intensity', 'observed', 'relative', 'initial', 'x-ray', 'beam', 'that', 'angle', 'corresponds', 'distance', 'atoms', 'sample', 'case', 'usually', 'corresponds', 'distance', 'parallel', 'polymer', 'chains', 'sometimes', 'one', 'peak', 'example', 'simple', 'one', 'start', 'the', 'relationship', 'scattering', 'angle', 'interatomic', 'distance', 'given', 'braggs', 'law', 'fundamental', 'starting', 'point', 'x-ray', 'crystallography', 'first', 'expressed', 'sir', 'william', 'bragg', 'son', 'sir', 'lawrence', 'bragg', 'the', 'law', 'states', '[nλ', '2dsin', 'θ', 'labelbragg1\\', 'n', 'integer', '(typically', 'assumed', '1)', 'lambda', 'wavelength', 'x-ray', 'used', '(commonly', '1541', 'å', 'although', 'others', 'possible)', 'distance', 'regularly', 'repeating', 'atoms', 'θ', 'scattering', 'angle', 'rearranging', 'equation', 'refbragg1,', 'find', 'dfrac1541{2\\sin', 'θ\\', 'in', 'words', 'distance', 'atoms', 'inversely', 'proportional', 'scattering', 'angle', 'the', 'greater', 'angle', 'smaller', 'distance', 'example', '(pageindex1\\)', 'in', 'following', 'cases', 'determine', 'sample', '(a', 'b)', 'higher', 'crystalline', 'content', 'crystalline', 'domains', 'provide', 'additional', 'strength', 'polymer', 'materials', 'the', 'strong', 'attraction', 'possible', 'closely-aligned', 'chains', 'results', 'long', 'segments', 'polymer', 'held', 'firmly', 'position', 'consequently', 'chain', 'flow', 'limited', 'material', 'becomes', 'rigid', 'sometimes', 'rigid', 'segments', 'polymer', 'deliberately', 'built', 'structure', 'for', 'example', 'block', 'co-polymers', 'softer', 'flexible', 'blocks', 'often', 'paired', 'harder', 'rigid', 'blocks', 'the', 'soft', 'segments', 'may', 'greater', 'conformational', 'flexibility', 'weaker', 'intermolecular', 'attractions', 'the', 'hard', 'segments', 'may', 'conformationally', 'rigid', 'may', 'stronger', 'intermolecular', 'attractions', 'strong', 'dipoles', 'hydrogen', 'bonds', 'if', 'block', 'lengths', 'right', 'size', 'two', 'segments', 'able', 'separate', 'two', 'phases', 'as', 'result', 'stronger', 'intermolecular', 'attractions', 'lengths', 'chains', 'containing', 'hard', 'segments', 'cluster', 'together', 'pushing', 'soft', 'segments', 'would', 'otherwise', 'get', 'way', 'intermolecular', 'attractions', 'this', 'phenomenon', 'called', 'microphase', 'separation', 'the', 'result', 'material', 'contains', 'islands', 'strength', 'rigidity', 'matrix', 'flexible', 'polymer', 'chains', 'that', 'useful', 'combination', 'the', 'flexible', 'chains', 'soft', 'segments', 'allow', 'polymer', 'distorted', 'bent', 'compressed', 'hard', 'segments', 'put', 'limits', 'flexibility', 'keeping', 'material', 'firmly', 'together', 'because', 'usually', 'dealing', 'large', 'numbers', 'enchained', 'monomers', 'difference', 'two', 'kinds', 'segments', 'need', 'even', 'dramatic', 'a', 'copolymer', 'butadiene', 'styrene', 'hydrocarbons', 'form', 'microphase', 'separated', 'materials', 'in', 'case', 'intermolecular', 'attractions', 'dominated', 'weak', 'london', 'dispersion', 'forces', 'aromatic', 'groups', 'styrene', 'delocalized', 'pi', 'systems', 'london', 'dispersion', 'forces', 'slightly', 'stronger', 'as', 'result', 'polystyrene', 'blocks', 'cluster', 'together', 'surrounded', 'softer', 'polybutadiene', 'blocks', 'problem', 'pp61', 'identify', 'hard', 'segment', 'soft', 'segment', 'following', 'block-co-polymers', 'sometimes', 'separation', 'phases', 'directly', 'observed', 'via', 'microscopy', 'tunneling', 'electron', 'microscopy', '(tem)', 'technique', 'generate', 'images', 'cross-sectional', 'slice', 'material', 'the', 'material', 'generally', 'stained', 'heavy', 'metal', 'osmium', 'binds', 'preferentially', 'one', 'phase', 'the', 'stained', 'phase', 'shows', 'darker', 'tem', 'phase', 'isnt', 'stained', 'x-ray', 'diffraction', 'techniques', 'often', 'used', 'measure', 'distances', 'hard', 'segments', 'small-angle', 'x-ray', 'scattering', '(saxs)', 'similar', 'wide-angle', 'x-ray', 'scattering', '(waxs)', 'because', 'inverse', 'relationship', 'scattering', 'angle', 'distance', 'saxs', 'used', 'probe', 'regularly', 'repeating', 'structures', 'greater', 'distances', 'seen', 'waxs', 'that', 'makes', 'possible', 'see', 'peaks', 'hard', 'segments', 'distributed', 'regularly', 'enough', 'within', 'soft', 'matrix', 'note', 'saxs', 'x-axis', 'usually', 'labeled', 'q', 'scattering', 'vector', 'q', '4πsinθ', 'λ', 'but', 'since', '2sinθλ', 'q', '2πd', '2πq', 'that', 'gives', 'us', 'pretty', 'straightforward', 'way', 'calculating', 'distances', 'regularly-spaced', 'hard', 'segments', '(or', 'regularly-spaced', 'objects)', 'once', 'waxs', 'inverse', 'relationship', 'quantity', 'shown', 'x-axis', 'distances', 'space', 'problem', 'pp62', 'calculate', 'approximate', 'distances', 'revealed', 'following', 'saxs', 'results', 'if', 'think', 'materials', 'made', 'polymers', 'couple', 'things', 'might', 'readily', 'come', 'mind', 'you', 'might', 'think', 'ubiquitous', 'use', 'plastics', 'structures', 'around', 'us', 'automobile', 'bumpers', 'you', 'might', 'think', 'rubber', 'band', 'use', 'fasten', 'something', 'firmly', 'either', 'home', 'lab', 'in', 'either', 'case', 'function', 'material', 'relies', 'response', 'stimuli', 'do', 'want', 'flexible', 'change', 'shape', 'snap', 'back', 'came', 'exactly', 'much', 'flexibility', 'need', 'where', 'trade-off', 'flexibility', 'strength', 'these', 'questions', 'important', 'polymer', 'chemistry', 'consequently', 'often', 'need', 'probe', 'materials', 'behave', 'different', 'conditions', 'know', 'employed', 'effectively', 'tensile', 'testing', 'one', 'simplest', 'ways', 'probe', 'material', 'responds', 'stress', 'remember', 'stress', 'context', 'means', 'force', 'exerted', 'material', 'per', 'unit', 'cross-sectional', 'area', 'as', 'units', 'pressure', 'pascals', '(pa)', 'to', 'experiment', 'two', 'ends', 'sample', 'attached', 'two', 'clamps', 'one', 'movable', 'the', 'movable', 'clamp', 'pulled', 'sample', 'becomes', 'stretched', 'the', 'force', 'required', 'pull', 'sample', 'recorded', 'given', 'cross-sectional', 'area', 'sample', 'perpendicular', 'force', 'force', 'converted', 'units', 'strain', 'the', 'experiment', 'also', 'measures', 'distance', 'sample', 'stretched', '(the', 'strain)', 'that', 'distance', 'usually', 'expressed', 'ratio', 'comparing', 'change', 'length', 'original', 'length', 'sample', 'it', 'written', 'either', 'fraction', 'percent', 'in', 'rough', 'sketch', 'see', 'typically', 'happens', 'experiment', 'the', 'sample', 'stretches', '(the', 'strain', 'increases)', 'gets', 'harder', 'stretch', 'seen', 'increasing', 'force', '(and', 'therefore', 'increasing', 'stress)', 'needed', 'keep', 'stretching', 'eventually', 'stress', 'plummets', 'sample', 'breaks', 'already', 'graph', 'see', 'couple', 'important', 'pieces', 'information', 'learn', 'tensile', 'testing', 'number', 'one', 'far', 'stretch', 'material', 'breaks', 'that', 'quantity', 'called', 'strain', 'break', 'in', 'sample', 'strain', 'break', 'looks', 'like', '11', '110;', 'means', 'sample', 'stretched', 'twice', 'original', 'length', 'number', 'two', 'much', 'stress', 'sample', 'support', 'without', 'breaking', 'that', 'quantity', 'called', 'ultimate', 'tensile', 'strength.', 'in', 'sample', 'ultimate', 'tensile', 'strength', '750', 'pa.', 'if', 'look', 'closely', 'graph', 'another', 'sample', 'get', 'third', 'important', 'quantity', 'see', 'additional', 'features', 'the', 'important', 'quantity', 'derived', 'first', 'part', 'curve', '(a)', 'stress', 'increases', 'linearly', 'strain', 'in', 'linear', 'region', 'material', 'behaving', 'hookean', 'solid', 'meaning', 'obeys', 'hookes', 'law', 'hookes', 'law', 'says', 'stress', 'strain', 'directly', 'proportional', 'in', 'original', 'words,', 'extension', 'proportional', 'force', 'f', 'kx', 'f', 'force', 'x', 'extension', 'solid', 'k', 'proportionality', 'constant', 'hookes', 'law', 'commonly', 'applied', 'behavior', 'mechanical', 'springs', 'also', 'holds', 'solid', 'materials', 'the', 'slope', 'linear', 'region', 'graph', '(a)', 'would', 'equal', 'proportionality', 'constant', 'k', 'graph', 'shows', 'ratio', 'fx', 'in', 'materials', 'science', 'slope', 'commonly', 'called', 'youngs', 'modulus', 'it', 'measure', 'inherent', 'stiffness', 'material.', 'youngs', 'modulus', 'e', 'σε', 'σ', 'stress', 'ε', 'extensional', 'strain', 'the', 'extensional', 'strain', 'strain', 'observed', 'stretching', 'material', 'in', 'graph', 'shown', 'youngs', 'modulus', 'around', '250', 'pa', '01', '2500', 'pa', 'this', 'initial', 'region', 'curve', 'hookes', 'law', 'obeyed', 'sometimes', 'called', 'linear', 'elastic', 'region', 'the', 'word', 'elastic', 'immediate', 'connotation', 'everyday', 'english', 'bringing', 'mind', 'rubber', 'band', 'stretched', 'think', 'stretching', 'part', 'hear', 'word', 'elastic', 'however', 'returning', 'motion', 'essential', 'feature', 'elastic', 'behavior', 'the', 'rubber', 'band', 'always', 'comes', 'back', 'original', 'shape.', 'likewise', 'within', 'linear', 'elastic', 'region', 'solid', 'material', 'returns', 'original', 'shape', 'deformed', 'stress.', 'with', 'solids', 'aluminum', 'concrete', 'linear', 'elastic', 'region', 'spans', 'narrow', 'range', 'strains', 'just', 'looking', 'wouldnt', 'notice', 'materials', 'deformed', 'however', 'linear', 'stress-strain', 'relationship', 'typical', 'solids', 'in', 'rheology', 'behavior', 'described', 'elastic', 'of', 'course', 'many', 'polymers', 'sample', 'stretched', 'far', 'see', 'change', 'naked', 'eye', 'still', 'returns', 'original', 'shape', 'what', 'makes', 'polymers', 'different', 'the', 'long-chain', 'structure', 'polymers', 'make', 'behave', 'differently', 'materials', 'these', 'chains', 'undergo', 'conformational', 'change', 'bond', 'along', 'chain', 'rotate', 'converting', 'polymer', 'chain', 'slightly', 'different', 'shape', 'that', 'ability', 'gives', 'soft', 'material', 'great', 'deal', 'flexibility', 'the', 'conformations', 'chains', 'adapt', 'accommodate', 'stress', 'moving', 'chains', 'new', 'shapes', 'offer', 'lower-energy', 'packing', 'when', 'stress', 'removed', 'chains', 'eventually', 'slide', 'back', 'original', 'conformations', 'they', 'return', 'equilibrium', 'shapes', 'the', 'presence', 'physical', 'chemical', 'crosslinks', 'help', 'material', 'return', 'original', 'shape', 'functioning', 'anchor', 'points', 'chains', 'dont', 'stray', 'far', 'those', 'interactions', 'might', 'found', 'hard-phase', 'interactions', 'microphase', 'separated', 'material', 'illustrated', 'although', 'could', 'also', 'found', 'homogeneous', 'material', 'the', 'linear', 'elastic', 'region', 'isnt', 'see', 'stress-strain', 'curve', 'at', 'point', '(b)', 'linear', 'relationship', 'suddenly', 'lost', 'the', 'stress', 'might', 'even', 'drop', 'seen', 'particular', 'case', 'this', 'feature', 'graph', 'called', 'yield', 'point', 'the', 'stress', 'experienced', 'material', 'resulting', 'strain', 'become', 'sufficient', 'overcome', 'natural', 'elastic', 'behavior', 'solid', 'as', 'noted', 'previously', 'physical', 'crosslinks', 'hydrogen', 'bonds', 'help', 'reinforce', 'elastic', 'behavior', 'polymer', 'sample', 'if', 'point', 'interactions', 'overcome', 'chains', 'start', 'slide', 'readily', 'past', 'other.', 'as', 'result', 'material', 'loses', 'elasticity', 'when', 'stress', 'released', 'material', 'still', 'spring', 'back', 'chains', 'settle', 'new', 'conformational', 'equilibrium', 'however', 'equilibrium', 'one', 'new', 'physical', 'crosslinks', 'form', 'groups', 'form', 'intermolecular', 'interactions', 'nearest', 'neighbors', 'encounter', 'it', 'isnt', 'likely', 'always', 'groups', 'interacting', 'as', 'result', 'material', 'settle', 'slightly', 'different', 'shape', 'you', 'probably', 'seen', 'happen', 'rubber', 'band', 'stretched', 'far', 'often', 'long', 'chains', 'dropped', 'old', 'interactions', 'picked', 'new', 'ones', 'formed', 'easily', 'extended', 'shape', 'at', 'point', '(c)', 'example', 'graph', 'slope', 'curve', 'starts', 'increase', 'the', 'change', 'stress', 'results', 'smaller', 'smaller', 'changes', 'strain', 'material', 'getting', 'stiffer', 'this', 'phenomenon', 'called', 'strain', 'hardening', 'this', 'feature', 'would', 'always', 'observed', 'happen', 'would', 'explain', 'in', 'polymers', 'one', 'explanation', 'may', 'lay', 'fact', 'volume', 'material', 'remain', 'constant', 'stretched', 'if', 'sample', 'getting', 'longer', 'means', 'also', 'getting', 'narrower.', 'as', 'consequence', 'narrowing', 'cross-section', 'chains', 'become', 'compressed', 'together', 'at', 'point', 'physical', 'crosslinks', 'begin', 'occurring', 'neighboring', 'chains', 'these', 'crosslinks', 'dont', 'occur', 'equilibrium', 'positions', 'polymer', 'chains', 'coiled', 'around', 'like', 'original', 'sample', 'these', 'crosslinks', 'occur', 'chains', 'extended', 'lying', 'parallel', 'closer', 'contact', 'distances', 'normal', 'we', 'see', 'opposite', 'sort', 'thing', 'happening', '(d)', 'slope', 'curve', 'decreasing', 'instead', 'increasing', 'this', 'phenomenon', 'called', 'strain-softening', 'again', 'might', 'observed', 'samples', 'when', 'happen', 'causes', 'in', 'case', 'answer', 'simpler', 'having', 'overcome', 'interactions', 'held', 'chains', 'together', 'nothing', 'left', 'resist', 'deformation', 'as', 'chains', 'begin', 'disentangle', 'becomes', 'even', 'easier', 'pull', 'apart', 'facilitating', 'extension', 'stretching', 'sample.', 'eventually', 'breaking', 'point', '(e)', 'chains', 'start', 'lose', 'contact', 'location', 'sample', 'resulting', 'catastrophic', 'rupture', 'sample', 'problem', 'pp71', 'in', 'following', 'curves', 'estimate', 'ultimate', 'tensile', 'strength', 'strain', 'break', 'problem', 'pp72', 'in', 'following', 'curves', 'calculate', 'youngs', 'modulus', 'problem', 'pp73', 'in', 'following', 'curves', 'identify', 'diagnostic', 'features', 'we', 'saw', 'earlier', 'inherent', 'stiffness', 'material', 'assessed', 'youngs', 'modulus', 'the', 'youngs', 'modulus', 'ratio', 'stress-induced', 'material', 'applied', 'strain', 'the', 'stress', 'amount', 'deformation', 'material', 'change', 'length', 'extensional', 'experiment', 'expressed', 'fraction', 'beginning', 'length', 'the', 'strain', 'force', 'exerted', 'sample', 'divided', 'cross-sectional', 'area', 'sample', 'if', 'strain', 'limited', 'small', 'deformation', 'varies', 'linearly', 'stress', 'if', 'graph', 'relationship', 'slope', 'line', 'gives', 'us', 'youngs', 'modulus', 'e', 'thats', 'proportionality', 'constant', 'stress', 'strain', 'hookes', 'law', 'e', 'σ', 'ε', 'hookes', 'law', 'sometimes', 'used', 'describe', 'behavior', 'mechanical', 'springs', 'the', 'modulus', 'thought', 'resistance', 'stretching', 'spring', 'resistance', 'spring', 'offers', 'greater', 'force', 'needed', 'stretch', 'the', 'force', 'snaps', 'spring', 'back', 'place', 'let', 'go', 'in', 'experiments', 'saw', 'earlier', 'didnt', 'let', 'go', 'we', 'continued', 'stretch', 'material', 'farther', 'farther', 'applying', 'generally', 'increasing', 'stress', 'material', 'finally', 'broke', 'now', 'look', 'much', 'limited', 'approach', 'instead', 'stretching', 'material', 'far', 'stretch', 'tiny', 'bit', 'release', 'stress', 'snaps', 'back', 'original', 'length', 'we', 'stress', 'release', 'we', 'keep', 'repeating', 'instead', 'continuously', 'increasing', 'strain', 'sample', 'subjected', 'oscillatory', 'strain', 'one', 'repeats', 'cycle', 'this', 'approach', 'called', 'dynamic', 'mechanical', 'analysis', 'we', 'use', 'dynamic', 'mechanical', 'analysis', 'measure', 'modulus', 'material', 'instead', 'continuously', 'moving', 'way', 'linear', 'elastic', 'region', 'beyond', 'hookes', 'law', 'breaks', 'carefully', 'keep', 'sample', 'hookean', 'region', 'entire', 'experiment', 'now', 'one', 'experiment', 'good', 'enough', 'extract', 'modulus', 'letting', 'go', 'why', 'the', 'principle', 'reason', 'running', 'experiment', 'way', 'get', 'additional', 'information', 'we', 'get', 'information', 'polymers', 'dont', 'quite', 'follow', 'hookes', 'law', 'perfectly', 'in', 'reality', 'even', 'within', 'linear', 'elastic', 'region', 'stress-strain', 'curve', 'quite', 'linear', 'in', 'picture', 'curvature', 'exaggerated', 'quite', 'bit', 'illustrative', 'purposes', 'even', 'relationship', 'quite', 'linear', 'release', 'strain', 'stress', 'material', 'simply', 'follow', 'curve', 'back', 'zero', 'it', 'instead', 'phenomenon', 'called', 'hysteresis', 'work', 'hysteresis', 'means', 'property', 'material', 'depends', 'material', 'came', 'current', 'situation', 'in', 'case', 'hookes', 'law', 'seems', 'imply', 'specific', 'sample', 'subjected', 'specific', 'strain', 'would', 'experience', 'specific', 'stress', '(or', 'vice', 'versa)', 'however', 'depends', 'whether', 'stretching', 'sample', 'letting', 'relax', 'as', 'let', 'sample', 'relax', 'back', 'initial', 'length', 'strain', 'different', 'saw', 'stretching', 'typically', 'lower', 'again', 'see', 'curve', 'curvature', 'exaggerated', 'the', 'difference', 'loading', 'curve', '(when', 'stress', 'first', 'applied)', 'unloading', 'curve', '(when', 'stress', 'removed)', 'represents', 'energy', 'loss', 'a', 'force', 'applied', 'move', 'sample', 'portion', 'sample', 'distance', 'when', 'sample', 'snapped', 'back', 'distance', 'force', 'unequal', 'one', 'initially', 'applied', 'some', 'energy', 'therefore', 'lost', 'the', 'slope', 'loading', 'curve', 'analogous', 'youngs', 'modulus', 'tensile', 'testing', 'experiment', 'called', 'storage', 'modulus', 'e', 'the', 'storage', 'modulus', 'measure', 'much', 'energy', 'must', 'put', 'sample', 'order', 'distort', 'the', 'difference', 'loading', 'unloading', 'curves', 'called', 'loss', 'modulus', 'e.', 'it', 'measures', 'energy', 'lost', 'cycling', 'strain.', 'why', 'would', 'energy', 'lost', 'experiment', 'in', 'polymer', 'chiefly', 'chain', 'flow', 'the', 'resistance', 'deformation', 'polymer', 'comes', 'entanglement', 'including', 'physical', 'crosslinks', 'general', 'occlusions', 'chains', 'encounter', 'undergoing', 'conformational', 'changes', 'accommodate', 'new', 'shape', 'material', 'once', 'stress', 'removed', 'material', 'springs', 'back', 'equilibrium', 'shape', 'reason', 'chains', 'would', 'follow', 'exact', 'conformational', 'pathway', 'return', 'equilibrium', 'conformations', 'because', 'moved', 'original', 'positions', 'able', 'follow', 'lower-energy', 'pathway', 'back', 'starting', 'point', 'pathway', 'less', 'resistance', 'neighboring', 'chains', 'for', 'reason', 'stretching', 'polymer', 'quite', 'stretching', 'mechanical', 'spring', 'a', 'spring-and-dashpot', 'analogy', 'often', 'invoked', 'describe', 'soft', 'materials', 'whereas', 'spring', 'simply', 'bounces', 'back', 'original', 'shape', 'pulled', 'dashpot', 'if', 'dont', 'know', 'dashpot', 'picture', 'hydraulic', 'arms', 'support', 'hatchback', 'car', 'open', 'upward', 'there', 'resistance', 'opening', 'hatchback', 'piston', 'pulled', 'hydraulic', 'fluid', 'arm', 'stretches', 'when', 'stop', 'lifting', 'arms', 'stay', 'length', 'hydraulic', 'fluid', 'also', 'resists', 'movement', 'piston', 'back', 'original', 'position', 'the', 'dashpot', 'tendency', 'stay', 'put', 'rather', 'spring', 'back', 'polymers', 'display', 'little', 'properties', 'they', 'elastic', 'element', 'rooted', 'entanglement', 'makes', 'resist', 'deformation', 'return', 'original', 'shapes', 'they', 'also', 'viscous', 'element', 'rooted', 'chain', 'flow', 'that', 'viscous', 'element', 'means', 'distort', 'polymeric', 'materials', 'might', 'return', 'exactly', 'form', 'started', 'taken', 'together', 'behaviors', 'described', 'viscoelastic', 'properties', 'many', 'materials', 'viscoelastic', 'properties', 'meaning', 'display', 'aspects', 'elastic', 'solids', 'aspects', 'viscous', 'liquids', 'so', 'far', 'concentrated', 'extensional', 'deformations', 'materials', 'looking', 'happens', 'stretch', 'its', 'worth', 'looking', 'another', 'type', 'deformation', 'commonly', 'used', 'materials', 'testing', 'this', 'second', 'approach', 'uses', 'shear', 'instead', 'extension', 'probe', 'material', 'respond', 'a', 'shear', 'force', 'applied', 'unevenly', 'material', 'tilts', 'twists', 'rather', 'stretching', 'one', 'reasons', 'approach', 'used', 'often', 'easy', 'a', 'sample', 'sandwiched', 'two', 'plates', 'the', 'bottom', 'plate', 'held', 'place', 'top', 'plate', 'twisted', 'shearing', 'material', 'held', 'if', 'take', 'closer', 'look', 'layer', 'sample', 'maybe', 'surface', 'along', 'edge', 'sandwich', 'imagine', 'breaking', 'individual', 'layers', 'under', 'shear', 'strain', 'layers', 'move', 'different', 'amounts', 'the', 'top', 'layer', 'right', 'beneath', 'top', 'plate', 'moves', 'the', 'bottom', 'layer', 'sitting', 'stationary', 'lower', 'plate', 'doesnt', 'move', 'in', 'layer', 'moves', 'little', 'one', 'beneath', 'this', 'gradation', 'deformation', 'across', 'sample', 'much', 'like', 'saw', 'analysis', 'viscosity', 'liquids', 'the', 'difference', 'viscosity', 'looks', 'variation', 'strain', 'time', 'nevertheless', 'modulus', 'solids', 'roughly', 'analogous', 'viscosity', 'liquids', 'we', 'use', 'parallel', 'plate', 'geometry', 'obtain', 'values', 'storage', 'modulus', 'loss', 'modulus', 'like', 'via', 'extensional', 'geometry', 'the', 'values', 'get', 'quite', 'for', 'reason', 'modulus', 'obtained', 'shear', 'experiments', 'given', 'different', 'symbol', 'modulus', 'obtained', 'extensional', 'experiments', 'in', 'shear', 'experiment', 'g', 'σ', 'ε', 'that', 'means', 'storage', 'modulus', 'given', 'symbol', 'g', 'loss', 'modulus', 'given', 'symbol', 'g', 'apart', 'providing', 'little', 'information', 'experiment', 'actually', 'conducted', 'distinction', 'shear', 'modulus', 'extension', 'modulus', 'important', 'resulting', 'values', 'quite', 'different', 'in', 'general', 'value', 'storage', 'modulus', 'obtained', 'extensional', 'experiment', 'three', 'times', 'larger', 'value', 'storage', 'modulus', 'obtained', 'shear', 'experiment.', 'e', '3', 'g', 'the', 'reason', 'difference', 'extension', 'actually', 'involves', 'deformation', 'material', 'three', 'directions', 'as', 'material', 'stretched', 'one', 'direction', '(lets', 'say', 'y-direction)', 'order', 'preserve', 'constant', 'volume', 'material', '(there', 'still', 'amount', 'stuff', 'stretching)', 'material', 'compresses', 'two', 'directions', '(x', 'z)', 'problem', 'pp81', 'metric', 'prefixes', 'frequently', 'encountered', 'reading', 'modulus', 'rank', 'following', 'units', 'stress', 'smallest', 'largest', 'case', 'provide', 'conversion', 'factor', 'pa', 'gpa', 'kpa', 'mpa', 'pa', 'the', 'storage', 'modulus', 'measures', 'resistance', 'deformation', 'elastic', 'solid', 'its', 'related', 'proportionality', 'constant', 'stress', 'strain', 'hookes', 'law', 'states', 'extension', 'increases', 'force', 'in', 'dynamic', 'mechanical', 'analysis', 'look', 'stress', '(σ)', 'force', 'per', 'cross-sectional', 'unit', 'area', 'needed', 'cause', 'extension', 'sample', 'strain', '(ε)', 'e', 'σ', 'ε', 'alternatively', 'shear', 'experiment', 'g', 'σ', 'ε', 'the', 'dynamic', 'mechanical', 'analysis', 'differs', 'simple', 'tensile', 'testing', 'performing', 'experiment', 'cyclically', 'the', 'sample', 'stretched', 'released', '(or', 'sheared', 'released)', 'it', 'subjected', 'additional', 'stress', 'released', 'there', 'element', 'time', 'involved', 'specifically', 'experiment', 'cyclic', 'carried', 'different', 'frequencies', 'when', 'plot', 'resulting', 'modulus', 'frequency', 'get', 'additional', 'information', 'sample', 'the', 'results', 'would', 'typically', 'presented', 'graph', 'like', 'one', 'what', 'graph', 'tells', 'us', 'frequency', 'clearly', 'matters', 'when', 'experiment', 'run', 'higher', 'frequencies', 'storage', 'modulus', 'higher', 'the', 'material', 'appears', 'stiffer', 'in', 'contrast', 'loss', 'modulus', 'lower', 'high', 'frequencies', 'material', 'behaves', 'much', 'less', 'like', 'viscous', 'liquid', 'in', 'particular', 'sharp', 'drop', 'loss', 'modulus', 'related', 'relaxation', 'time', 'material', 'in', 'context', 'thats', 'time', 'takes', 'chains', 'flow', 'new', 'conformations', 'response', 'applied', 'stress', 'if', 'dont', 'time', 'flow', 'viscous', 'response', 'material', 'lost', 'the', 'material', 'behaves', 'much', 'like', 'elastic', 'solid', 'subjected', 'high-frequency', 'cyclic', 'deformation', 'thats', 'important', 'know', 'material', 'might', 'subjected', 'vibrations', 'stimuli', 'everyday', 'use', 'properties', 'might', 'change', 'accordingly', 'another', 'variation', 'kind', 'experiment', 'called', 'dynamic', 'mechanical', 'thermal', 'analysis', 'instead', 'changing', 'frequency', 'stimulus', 'throughout', 'experiment', 'frequency', 'held', 'constant', 'temperature', 'changed', 'instead', 'as', 'result', 'see', 'material', 'responds', 'different', 'conditions', 'might', 'tell', 'us', 'behave', 'everyday', 'applications', 'the', 'result', 'experiment', 'might', 'graph', 'like', 'one', 'at', 'point', 'already', 'familiar', 'glass', 'transition', 'it', 'shouldnt', 'surprising', 'properties', 'dependent', 'temperature', 'at', 'glass', 'transition', 'temperature', 'expanding', 'volume', 'material', 'increasing', 'temperature', 'becomes', 'sufficient', 'allow', 'chain', 'flow', 'as', 'result', 'material', 'suddenly', 'behaves', 'much', 'like', 'viscous', 'liquid', 'loss', 'modulus', 'increases', 'the', 'stiffness', 'material', 'drops', 'entangled', 'chains', 'longer', 'resist', 'deformation', 'strongly', 'storage', 'modulus', 'decreases', 'the', 'dynamic', 'mechanical', 'thermal', 'analysis', 'thus', 'provides', 'alternative', 'way', 'determine', 'glass', 'transition', 'temperature', 'because', 'actually', 'measuring', 'different', 'physical', 'phenomenon', 'differential', 'scanning', 'calorimetry', 'tg', 'obtained', 'dmta', 'experiment', 'may', 'agree', 'exactly', 'one', 'obtained', 'dsc', 'experiment', 'nevertheless', 'often', 'useful', 'different', 'ways', 'assessing', 'properties', 'in', 'order', 'facilitate', 'analysis', 'tg', 'experiment', 'different', 'quantity', 'usually', 'displayed', 'tan', 'delta', 'ratio', 'loss', 'modulus', 'storage', 'modulus', 'it', 'peaks', 'glass', 'transition', 'temperature', 'the', 'term', 'tan', 'delta', 'refers', 'mathematical', 'treatment', 'storage', 'modulus', 'happens', 'in-phase', '(or', 'time', 'as)', 'application', 'stress', 'whereas', 'loss', 'modulus', 'happens', 'out-of-phase', 'application', 'stress', 'because', 'would', 'take', 'time', 'chains', 'move', 'new', 'confirmation', 'subjected', 'stress', 'strain', 'actually', 'lags', 'behind', 'stress', 'experiments', 'delta', 'refers', 'phase', 'lag', 'amount', 'time', 'application', 'stress', 'observation', 'maximum', 'strain', 'you', 'may', 'remember', 'sine', 'curve', 'cosine', 'curve', 'phase', 'storage', 'modulus', 'described', 'proportional', 'cosδ', 'whereas', 'loss', 'modulus', 'proportional', 'sinδ', 'the', 'ratio', 'cosδ', 'sinδ', 'tanδ', 'why', 'tanδ', 'peak', 'glass', 'transition', 'temperature', 'clearly', 'chains', 'begin', 'move', 'freely', 'loss', 'modulus', 'increases', 'consequently', 'material', 'also', 'becomes', 'less', 'stiff', 'rubbery', 'the', 'storage', 'modulus', 'drops', 'if', 'tan', 'delta', 'ratio', 'loss', 'modulus', 'storage', 'modulus', 'increase', 'point', '--', 'why', 'drop', 'thats', 'loss', 'modulus', 'refers', 'energy', 'loss', 'material', 'gotten', 'softer', 'less', 'stress', '(and', 'less', 'energy)', 'put', 'sample', 'first', 'place', 'energy', 'loss', 'also', 'gets', 'smaller', 'as', 'result', 'tan', 'delta', 'goes', 'glass', 'transition', 'drops', 'shortly', 'beyond', 'point', 'problem', 'pp91', 'estimate', 'storage', 'loss', 'modulus', 'glassy', 'phase', 'rubbery', 'phase', 'following', 'cases', 'problem', 'pp92', 'estimate', 'tg', 'following', 'cases', 'problem', 'pp11', 'a)', 'with', 'shorter', 'elution', 'time', 'i', 'higher', 'molecular', 'weight', 'ii', 'the', 'narrower', 'peak', 'means', 'i', 'narrower', 'dispersity', 'ii', 'b)', 'with', 'shorter', 'elution', 'time', 'iii', 'higher', 'molecular', 'weight', 'iv', 'the', 'wider', 'peak', 'means', 'iii', 'broader', 'dispersity', 'iv', 'c)', 'with', 'higher', 'elution', 'time', 'v', 'lower', 'molecular', 'weight', 'vi', 'the', 'wider', 'peak', 'means', 'v', 'broader', 'dispersity', 'vi', 'problem', 'pp12', 'a)', 'repeat', 'unit', '109', 'x', '(c4h6o2)', '109', 'x', '8609', 'gmol', '938381', 'gmol', 'end', 'groups', 'c9h11', 'br', '11919', '799', '19909', 'gmol', 'total', '938381', '19909', '958290', 'gmol', 'b)', 'repeat', 'unit', '763', 'x', '(c8h8)', '763', 'x', '10415', 'gmol', '7946645', 'gmol', 'end', 'groups', 'c4h9', 'h', '5712', '1008', '5813', 'gmol', 'total', '7946645', '5813', '7952458', 'gmol', 'c)', 'repeat', 'unit', '48', 'x', '(c4h6o2)', '48', 'x', '8609', 'gmol', '413232', 'gmol', 'end', 'groups', 'c4h9', 'c7h5s2', '5712', '15326', '21038', 'gmol', 'total', '413232', '21038', '434270', 'gmol', 'problem', 'pp13', 'the', 'ratio', 'repeat', 'unit', 'integral', 'per', 'proton', 'end', 'group', 'proton', 'per', 'integral', 'gives', 'degree', 'polymerization', 'we', 'could', 'take', 'entire', 'integration', 'end', 'group', 'divide', 'entire', 'number', 'protons', 'group', 'select', 'one', 'position', 'represent', 'end', 'group', 'similarly', 'select', 'one', 'position', 'represent', 'repeat', 'unit', 'a)', 'repeat', 'unit', 'integral', 'per', 'proton', '360', '2h', '18', 'end', 'group', 'integral', 'per', 'proton', '032', '2h', '016', 'degree', 'polymerization', '18016', '112', 'b)', 'repeat', 'unit', 'integral', 'per', 'proton', '260', '1h', '26', 'end', 'group', 'integral', 'per', 'proton', '132', '9h', '015', 'degree', 'polymerization', '26015', '173', 'c)', 'repeat', 'unit', 'integral', 'per', 'proton', '540', '4h', '135', 'end', 'group', 'integral', 'per', 'proton', '049', '1h', '049', 'degree', 'polymerization', '135049', '28', 'problem', 'pp21', 'α', 'slope', 'riserun', 'thats', 'approximately', '40-24/66-44', '16/22', '073.', 'k', 'y-intercept', 'the', 'equation', 'straight', 'line', 'mx', 'b', 'case', '073x', 'b', 'if', 'choose', 'point', 'line', '(xy)', '(49', '20)', 'substitute', 'values', 'x', 'get', 'b', 'so', '20', '073(49)', 'b', 'b', '20', '-', '356', '-156', 'problem', 'pp22', 'if', 'molecular', 'weight', 'million', 'gmol', 'log(mw)', '6', 'interpolating', 'log(η)', '4', 'η', '10000', 'mlg', 'problem', 'pp23', 'if', 'intrinsic', 'viscosity', 'η', '800', 'mlg', 'log(η)', '29', 'interpolating', 'log(mw)', '51', 'mw', '126000', 'gmol', 'problem', 'pp24', 'ethylene', 'glycol', 'form', 'hydrogen', 'bonds', 'either', 'end', 'molecule', 'forming', 'supramolecular', 'assembly', 'much', 'like', 'polymer', 'as', 'result', 'much', 'greater', 'drag', 'solution', 'higher', 'viscosity', 'problem', 'pp31', 'honey', 'concentrated', 'solution', 'simple', 'sugars', 'small', 'molecules', 'molasses', 'although', 'similar', 'honey', 'ways', 'also', 'contain', 'starches', 'polymers', 'this', 'polymeric', 'content', 'leads', 'shear-thinning', 'behavior', 'problem', 'pp41', 'a)', 'there', 'glass', 'transition', 'around', '-18°c', 'b)', 'there', 'melting', 'point', 'around', '125°c', 'c)', 'there', 'glass', 'transition', 'around', '-4°c', 'd)', 'there', 'glass', 'transition', 'around', '117°c', 'melting', 'point', 'around', '146°c', 'problem', 'pp51', 'a)', 'tg', 'observed', 'around', '78°c', 'tm', 'observed', 'around', '117°c', 'tc', 'observed', 'around', '104°c', 'b)', 'tg', 'observed', 'around', '134°c', 'tm', 'observed', 'around', '167°c', 'tc', 'observed', 'sample', 'failed', 'crystallize', 'remained', 'amorphous', 'solid', 'c)', 'tm', 'observed', 'around', '194°c', 'tc', 'observed', 'around', '187°c', 'tg', 'observed', 'probably', 'occurs', '150°c', 'd)', 'tg', 'observed', 'around', '123°c', 'tm', 'observed', 'the', 'experiment', 'checked', 'much', 'higher', 'tg', '(over', 'hundred', 'degrees)', 'material', 'may', 'amorphous', 'solid', 'problem', 'pp61', 'problem', 'pp62', 'a)', '(2', 'x', '314)', '040', '16', 'å', '(2', 'x', '314)', '070', '90', 'å', 'b)', '(2', 'x', '314)', '025', '25', 'å', '(2', 'x', '314)', '085', '74', 'å', 'c)', '(2', 'x', '314)', '025', '25', 'å', '(2', 'x', '314)', '052', '12', 'å', '(2', 'x', '314)', '066', '10', 'å', 'problem', 'pp71', 'a)', 'ultimate', 'tensile', 'strength', '800', 'pa', 'strain', 'break', '55', 'b)', 'ultimate', 'tensile', 'strength', '750', 'pa', 'strain', 'break', '215', 'c)', 'ultimate', 'tensile', 'strength', '220', 'pa', 'strain', 'break', '120', 'problem', 'pp72', 'a)', 'e', 'σ', 'ε', '180', 'pa', '030', '600', 'pa', 'b)', 'e', 'σ', 'ε', '450', 'pa', '015', '3000', 'pa', 'c)', 'e', 'σ', 'ε', '50', 'pa', '025', '200', 'pa', 'd)', 'e', 'σ', 'ε', '75', 'pa', '030', '250', 'pa', 'problem', 'pp73', 'problem', 'pp81', '1', 'pa', '1', 'pa', '1', 'kpa', '1000', 'pa', '1', 'mpa', '1000000', 'pa', '1gpa', '1000000000', 'pa', 'problem', 'pp91', 'a)', 'glassy', 'storage', 'modulus', '15', 'mpa', 'loss', 'modulus', '70', 'kpa', 'rubbery', 'storage', 'modulus', '7', 'mpa', 'loss', 'modulus', '80', 'kpa', 'b)', 'glassy', 'storage', 'modulus', '600', 'kpa', 'loss', 'modulus', '140', 'kpa', 'rubbery', 'storage', 'modulus', '130', 'kpa', 'loss', 'modulus', '150', 'kpa', 'c)', 'glassy', 'storage', 'modulus', '320', 'kpa', 'loss', 'modulus', '80', 'kpa', 'rubbery', 'storage', 'modulus', '70', 'kpa', 'loss', 'modulus', '70', 'kpa', 'problem', 'pp92', 'a)', '89', '°c', 'b)', '170', '°c', 'c)', '124', '°c']
pract_list = []
pract_list.append(test_schaller_list)

In [35]:
word2vec_book_model(pract_list)

INFO - 18:06:06: collecting all words and their counts
INFO - 18:06:06: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 18:06:06: collected 11219 word types from a corpus of 10237 words (unigram + bigrams) and 1 sentences
INFO - 18:06:06: using 11219 counts as vocab in Phrases<0 vocab, min_count=30, threshold=10.0, max_vocab_size=40000000>
INFO - 18:06:06: source_vocab length 11219
INFO - 18:06:06: Phraser built with 1 phrasegrams
INFO - 18:06:06: collecting all words and their counts
INFO - 18:06:06: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 18:06:06: collected 2516 word types from a corpus of 10201 raw words and 1 sentences
INFO - 18:06:06: Loading a fresh vocabulary
INFO - 18:06:06: effective_min_count=2 retains 1275 unique words (50% of original 2516, drops 1241)
INFO - 18:06:06: effective_min_count=2 leaves 8960 word corpus (87% of original 10201, drops 1241)
INFO - 18:06:06: deleting the raw counts dictionary of 2516 items
INFO -

Time to build vocab: 0.0 mins


INFO - 18:06:06: training model with 6 workers on 1275 vocabulary and 300 features, using sg=0 hs=0 sample=6e-05 negative=0 window=2
INFO - 18:06:07: worker thread finished; awaiting finish of 5 more threads
INFO - 18:06:07: worker thread finished; awaiting finish of 4 more threads
INFO - 18:06:07: worker thread finished; awaiting finish of 3 more threads
INFO - 18:06:07: worker thread finished; awaiting finish of 2 more threads
INFO - 18:06:07: worker thread finished; awaiting finish of 1 more threads
INFO - 18:06:07: worker thread finished; awaiting finish of 0 more threads
INFO - 18:06:07: EPOCH - 1 : training on 10201 raw words (2929 effective words) took 0.0s, 193796 effective words/s
INFO - 18:06:07: worker thread finished; awaiting finish of 5 more threads
INFO - 18:06:07: worker thread finished; awaiting finish of 4 more threads
INFO - 18:06:07: worker thread finished; awaiting finish of 3 more threads
INFO - 18:06:07: worker thread finished; awaiting finish of 2 more threads
I

Time to train the model: 0.03 mins


INFO - 18:06:08: precomputing L2-norms of word weight vectors


the size of the vocabulary is:  1275
[('known', 0.17378315329551697), ('state', 0.16282540559768677), ('neighbors', 0.14923037588596344), ('pont', 0.1468380242586136), ('025', 0.1459658294916153), ('experienced', 0.14523988962173462), ('concentration', 0.1419733166694641), ('mn', 0.14127886295318604), ('ponts', 0.13880686461925507), ('response', 0.13849309086799622)]
